In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from seaborn import violinplot
from seaborn import boxplot
from seaborn import histplot
from seaborn import countplot

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
import lightgbm as lgbm

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

##### O DataSet a ser analisado será o "Performance de estudantes", onde reúne alguns parâmetros de à classe social e notas em diferentes provas (matemática e português) de alunos reais de duas escolas brasileiras (Gabriel Pereira e Mousinho da Silveira), no ano de 2008.

##### O objetivo desta análise é entender melhor quais fatores podem contribuir mais para um determinado aluno conseguir ter um desempenho e aprendizado melhor, assim como poder prever, com um modelo simples, as notas mais prováveis de um aluno ter quando preenchendo alguns parâmetros.

##### Este DataSet foi retirado do website [UC Irvine Machine Learning Repository](https://archive.ics.uci.edu), e pode ser acessado por este [LINK](https://archive.ics.uci.edu/dataset/320/student+performance).

### Sumário

>- 1.....................................................Importação dos DataSets.
>
>   - 1.1........................................Definição de Funções.
>    
>- 2.....................................................Dicionário de Colunas.
>
>- 3.....................................................Preparação dos Dados.
>
>- 4.....................................................Uma análise exploratória.
>
>   - 4.1........................................Algumas análises Univariadas.
>
>   - 4.2........................................Algumas análises Bivariadas.
>
>       - 4.2.1...........................Relações de Nota vs Algumas variáveis categóricas.
>
>       - 4.2.2...........................Correlações das Variáveis numéricas.
>
>       - 4.2.3...........................Comportamento das médias.
>
>- 5.....................................................Considerações finais.
>
>   - 5.1........................................Conclusões e hipóteses.
>
>   - 5.2........................................Propostas.

 
>- 7.....................................................Planejamento dos Modelos.
>
>   - 7.1........................................O que queremos prever de fato ?
>
>   - 7.2........................................O que será feito, e como ?
>
>   - 7.3........................................Expectativa e próximos passos.
>
>- 8.....................................................Relação da EDA com a escolha das variáveis.
>
>- 9.....................................................Implementação dos Modelos.
>
>   - 9.1........................................Regressão Linear.
>
>   - 9.2........................................Árvore de Decisão.
>
>   - 9.3........................................KNN (K-Vizinhos mais Próximos).
>
>   - 9.4........................................Gradient Boosting.
>
>   - 9.5........................................Light GBM.
>
>- 10.....................................................Resultados e Métricas.

### Sumário de Imagens

> Gráfico 1...............................................Barras - Quantidade de cada gênero.
>
> Gráfico 2...............................................Histograma - Frequência de cada nota por trimestre.
>
> Gráfico 3...............................................Boxplot - Outliers de falta.
>
> Gráfico 4...............................................Violino - Frequência de Notas por Escola
>
> Gráfico 5...............................................Violino - Frequência de Notas por tipo de endereço.
>
> Gráfico 6...............................................Violino - Frequência de Notas por tempo de deslocamento.
>
> Gráfico 7...............................................Violino - Frequência de Notas por suporte educacional.
>
> Gráfico 8...............................................Violino - Frequência de Notas por relação familiar.
>
> Gráfico 9...............................................Violino - Frequência de Notas por tamanho da família.
>
> Gráfico 10...............................................Mapa de Calor - Correlação das variáveis numéricas de Matemática.
>
> Gráfico 11............................................Mapa de Calor - Correlação das variáveis numéricas de Português.
>
> Gráfico 12............................................Linha - Tempo de Estudo por Média de nota.
>
> Gráfico 13............................................Barras e Linha - Quantidade de Cada educação de Pai/Mãe e Média de nota por Educação dos pais.

> Imagem 14...............................................Mapa de Calor - Correlação das notas de Matemática com escola, internet e ingresso em ensino superior.
>
> Imagem 15...............................................Mapa de Calor - Correlação das notas de Português com escola, internet e ingresso em ensino superior.
>
> Imagem 16...............................................Tabela - Resultados e métricas dos Modelos treinados.

# 1 - Importação dos DataSets

##### Após fazer a importação dos datasets, podemos ver qual a cara dos dados, como estão escritas variáveis categóricas, se há caracteres especiais, assim como utilizar funções que nos dê uma idéia de dados faltantes.

##### No caso, podemos contar que estes datasets tem 1000 registros no total, e nenhum dos dados está nulo ou faltante, facilitando assim os próximos passos.

In [ ]:
df_mat = pd.read_csv("data/student-mat.csv", sep=';')
df_por = pd.read_csv("data/student-por.csv", sep=';')
dfs = [df_mat, df_por]

In [ ]:
df_mat.head(7)

In [ ]:
df_por.head(7)

In [ ]:
print("df_mat Info\n")
print(df_mat.info())
print("\n\ndf_por Info\n")
print(df_por.info())

In [ ]:
print("df_mat Quantidade Nulos\n")
print(df_mat.isnull().sum())
print("\n\ndf_por Quantidade Nulos\n")
print(df_por.isnull().sum())

### 1.1 Definição de Funções

In [ ]:
def showViolinPlot(var):
    sns.set_style('whitegrid')
    fig, axes = plt.subplots(2, 3, figsize=(12, 6))

    sns.violinplot(x=var, y='G1', data=df_mat, ax=axes[0][0])
    sns.violinplot(x=var, y='G2', data=df_mat, ax=axes[0][1])
    sns.violinplot(x=var, y='G3', data=df_mat, ax=axes[0][2])

    sns.violinplot(x=var, y='G1', data=df_por, ax=axes[1][0])
    sns.violinplot(x=var, y='G2', data=df_por, ax=axes[1][1])
    sns.violinplot(x=var, y='G3', data=df_por, ax=axes[1][2])

    for i in range(0, 2):
        for j in range(0, 3):
            label = axes[i][j].get_xticklabels()
            ticks = axes[i][j].get_xticks()
            axes[i][j].set_xticks(ticks)
            axes[i][j].set_xlabel("")
            axes[i][j].set_xticklabels(label)

    axes[0][0].set_ylabel("Nota  1º  Trimestre  -  Matematica")
    axes[0][1].set_ylabel("Nota  2º  Trimestre  -  Matematica")
    axes[0][2].set_ylabel("Nota  3º  Trimestre  -  Matematica")
    axes[1][0].set_ylabel("Nota  1º  Trimestre  -  Portugues")
    axes[1][1].set_ylabel("Nota  2º  Trimestre  -  Portugues")
    axes[1][2].set_ylabel("Nota  3º  Trimestre  -  Portugues")
    
    plt.tight_layout(w_pad=4, h_pad=2)
    plt.show()

In [ ]:
def showHeatmap(df, subject, colorPallete):
    plt.figure(figsize=(14,9))
    num = df.select_dtypes(include=['float', 'int'])
    corr = num.corr()
    matrix = np.triu(corr)
    sns.heatmap(corr, annot=True, cmap=colorPallete, mask=matrix)
    plt.title("Mapa de Calor de Correlações - " + subject, fontsize=15)
    plt.xticks(rotation=35)
    plt.yticks(rotation=0)
    plt.show()

In [ ]:
def showGenderQty(df):

    bar = df['sex'].value_counts().reset_index()
    percentages = bar['count'] / bar['count'].sum() * 100

    plt.grid(False)

    return bar, percentages

In [ ]:
def pred_decision_tree(df, X, y):

    X_train, X_test, y_train, y_test = train_test_split(df[X], df[y], test_size=0.2, random_state=42)

    tree = DecisionTreeRegressor(random_state=42)
    tree.fit(X_train, y_train)
    Y_pred_tree = tree.predict(X_test)
    mse_tree = mean_squared_error(y_test, Y_pred_tree)

    return mse_tree

In [ ]:
def showHeatmap(df, subject, colorPallete, index):
    plt.figure(figsize=(10,6))
    num = df.select_dtypes(include=['float', 'int'])
    corr = num.corr()
    matrix = np.triu(corr)
    sns.heatmap(corr, annot=True, cmap=colorPallete, mask=matrix)
    plt.title("Mapa de Calor de Correlações - " + subject, fontsize=15)
    plt.xticks(rotation=35)
    plt.yticks(rotation=0)
    plt.show()
    print("\t\tImagem " + str(index) + " - Mapa de Calor - Correlação das notas de " + subject + " com escola, internet e ingresso em ensino superior.")
    print("\n")

In [ ]:
def pred_linear_regression(df, X, y):

    X_train, X_test, y_train, y_test = train_test_split(df[X], df[y], test_size=0.2, random_state=42)

    modelo = LinearRegression()
    modelo.fit(X_train, y_train)

    y_pred = modelo.predict(X_test)

    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    return mse, r2

In [ ]:
def pred_gradient_boosting_regression(df, X, y):

    X_train, X_test, y_train, y_test = train_test_split(df[X], df[y], test_size=0.2, random_state=42)

    gboost = GradientBoostingRegressor(n_estimators=100, random_state=42)
    gboost.fit(X_train, y_train)

    Y_pred_gboost = gboost.predict(X_test)

    mse_gboost = mean_squared_error(y_test, Y_pred_gboost)
    return mse_gboost

In [ ]:
def pred_knn_regression(df, X, y):

    X_train, X_test, y_train, y_test = train_test_split(df[X], df[y], test_size=0.2, random_state=42)

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    knn = KNeighborsRegressor(n_neighbors=5)
    knn.fit(X_train_scaled, y_train)

    Y_pred_knn = knn.predict(X_test_scaled)

    mse_knn = mean_squared_error(y_test, Y_pred_knn)
    return mse_knn

In [ ]:
def pred_light_gbm(df, X, y):

    X_train, X_test, y_train, y_test = train_test_split(df[X], df[y], test_size=0.2, random_state=42)

    lgb_model = lgbm.LGBMRegressor(n_estimators=100, random_state=42, verbosity=-1)
    lgb_model.fit(X_train, y_train)

    Y_pred_lgb = lgb_model.predict(X_test)

    mse_lgb = mean_squared_error(y_test, Y_pred_lgb)
    return mse_lgb

In [ ]:
df_names = ['Matemática', 'Português']

coefs = pd.DataFrame({
    'materia': [],
    'modelo': [],
    'x': [],
    'variavel_target': [],
    'mse': [],
    'r2': [],
})

In [ ]:
def pred_linear_regression(df, X, y):

    X_train, X_test, y_train, y_test = train_test_split(df[X], df[y], test_size=0.2, random_state=42)

    modelo = LinearRegression()
    modelo.fit(X_train, y_train)

    y_pred = modelo.predict(X_test)

    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    return mse, r2, mae

def pred_decision_tree(df, X, y):

    X_train, X_test, y_train, y_test = train_test_split(df[X], df[y], test_size=0.2, random_state=42)

    tree = DecisionTreeRegressor(random_state=42)
    tree.fit(X_train, y_train)
    Y_pred_tree = tree.predict(X_test)
    mse_tree = mean_squared_error(y_test, Y_pred_tree)
    mae_tree = mean_absolute_error(y_test, Y_pred_tree)

    return mse_tree, mae_tree

def pred_gradient_boosting_regression(df, X, y):

    X_train, X_test, y_train, y_test = train_test_split(df[X], df[y], test_size=0.2, random_state=42)

    gboost = GradientBoostingRegressor(n_estimators=100, random_state=42)
    gboost.fit(X_train, y_train)

    Y_pred_gboost = gboost.predict(X_test)

    mse_gboost = mean_squared_error(y_test, Y_pred_gboost)
    mae_gboost = mean_absolute_error(y_test, Y_pred_gboost)
    return mse_gboost, mae_gboost

def pred_knn_regression(df, X, y):

    X_train, X_test, y_train, y_test = train_test_split(df[X], df[y], test_size=0.2, random_state=42)

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    knn = KNeighborsRegressor(n_neighbors=5)
    knn.fit(X_train_scaled, y_train)

    Y_pred_knn = knn.predict(X_test_scaled)

    mse_knn = mean_squared_error(y_test, Y_pred_knn)
    mae_knn = mean_absolute_error(y_test, Y_pred_knn)
    return mse_knn, mae_knn

def pred_light_gbm(df, X, y):

    X_train, X_test, y_train, y_test = train_test_split(df[X], df[y], test_size=0.2, random_state=42)

    lgb_model = lgbm.LGBMRegressor(n_estimators=100, random_state=42, verbosity=-1)
    lgb_model.fit(X_train, y_train)

    Y_pred_lgb = lgb_model.predict(X_test)

    mse_lgb = mean_squared_error(y_test, Y_pred_lgb)
    mae_lgb = mean_absolute_error(y_test, Y_pred_lgb)
    return mse_lgb, mae_lgb

df_names = ['Matemática', 'Português']

coefs = pd.DataFrame({
    'materia': [],
    'modelo': [],
    'x': [],
    'variavel_target': [],
    'mse': [],
    'r2': [],
    'mae': []
})

df_mat = pd.read_csv("data/student-mat.csv", sep=';')
df_por = pd.read_csv("data/student-por.csv", sep=';')
dfs = [df_mat, df_por]

for i in (dfs):
    i['Pstatus'] = i['Pstatus'].str.replace("A", "S").str.replace("T", "J")
    i['sex'] = i['sex'].str.replace("M", "Masculino").str.replace("F", "Feminino")
    i['address'] = i['address'].str.replace("R", "Rural").str.replace("U", "Urbano")
    i['Mjob'] = i['Mjob'].str.replace("at_home", "Lar").str.replace("health", "Saude").str.replace("other", "Outro").str.replace("services", "Func. Publico").str.replace("teacher", "Professor")
    i['Fjob'] = i['Fjob'].str.replace("at_home", "Lar").str.replace("health", "Saude").str.replace("other", "Outro").str.replace("services", "Func. Publico").str.replace("teacher", "Professor")
    i['reason'] = i['reason'].str.replace("home", "Local").str.replace("reputation", "Reputacao").str.replace("course", "Curso").str.replace("other", "Outro")
    i['guardian'] = i['guardian'].str.replace("mother", "Mae").str.replace("father", "Pai").str.replace("other", "Outro")

    i['schoolsup'] = i['schoolsup'].str.replace("yes", "Sim").str.replace("no", "Nao")
    i['famsup'] = i['famsup'].str.replace("yes", "Sim").str.replace("no", "Nao")
    i['paid'] = i['paid'].str.replace("yes", "Sim").str.replace("no", "Nao")
    i['activities'] = i['activities'].str.replace("yes", "Sim").str.replace("no", "Nao")
    i['nursery'] = i['nursery'].str.replace("yes", "Sim").str.replace("no", "Nao")
    i['higher'] = i['higher'].str.replace("yes", "Sim").str.replace("no", "Nao")
    i['internet'] = i['internet'].str.replace("yes", "Sim").str.replace("no", "Nao")
    i['romantic'] = i['romantic'].str.replace("yes", "Sim").str.replace("no", "Nao")

dfs[0] = pd.get_dummies(df_mat, columns=['school', 'higher', 'internet'])
dfs[1] = pd.get_dummies(df_por, columns=['school', 'higher', 'internet'])

for i in dfs:
    i['school_GP'] = i['school_GP'].astype(int)
    i['school_MS'] = i['school_MS'].astype(int)
    i['higher_Nao'] = i['higher_Nao'].astype(int)
    i['higher_Sim'] = i['higher_Sim'].astype(int)
    i['internet_Nao'] = i['internet_Nao'].astype(int)
    i['internet_Sim'] = i['internet_Sim'].astype(int)

for name, i in zip(df_names, dfs):
    X = ['failures', 'studytime', 'Medu', 'Fedu', 'school_GP', 'school_MS', 'higher_Nao', 'higher_Sim', 'internet_Nao', 'internet_Sim']
    y = 'G1'
    mse_lr, r2_lr, mae_lr = pred_linear_regression(i, X, y)
    coefs.loc[len(coefs)] = [name, 'Regressao Linear', str(X), str(y), mse_lr, f'{r2_lr:.2f}', mae_lr]
for name, i in zip(df_names, dfs):
    X = ['failures', 'studytime', 'Medu', 'Fedu', 'school_GP', 'school_MS', 'higher_Nao', 'higher_Sim', 'internet_Nao', 'internet_Sim']
    y = 'G2'
    mse_lr, r2_lr, mae_lr = pred_linear_regression(i, X, y)
    coefs.loc[len(coefs)] = [name, 'Regressao Linear', str(X), str(y), mse_lr, f'{r2_lr:.2f}', mae_lr]
for name, i in zip(df_names, dfs):
    X = ['failures', 'studytime', 'Medu', 'Fedu', 'school_GP', 'school_MS', 'higher_Nao', 'higher_Sim', 'internet_Nao', 'internet_Sim']
    y = ['G1', 'G2']
    mse_lr, r2_lr, mae_lr = pred_linear_regression(i, X, y)
    coefs.loc[len(coefs)] = [name, 'Regressao Linear', str(X), str(y), mse_lr, f'{r2_lr:.2f}', mae_lr]
for name, i in zip(df_names, dfs):
    X = ['G1', 'G2']
    y = 'G3'
    mse_lr, r2_lr, mae_lr = pred_linear_regression(i, X, y)
    coefs.loc[len(coefs)] = [name, 'Regressao Linear', str(X), str(y), mse_lr, f'{r2_lr:.2f}', mae_lr]



for name, i in zip(df_names, dfs):
    X = ['failures', 'studytime', 'Medu', 'Fedu', 'school_GP', 'school_MS', 'higher_Nao', 'higher_Sim', 'internet_Nao', 'internet_Sim']
    y = 'G1'
    mse_tree, mae_tree = pred_decision_tree(i, X, y)
    coefs.loc[len(coefs)] = [name, 'Arvore de  Decisao', str(X), str(y), mse_tree, '-', mae_tree]
for name, i in zip(df_names, dfs):
    X = ['failures', 'studytime', 'Medu', 'Fedu', 'school_GP', 'school_MS', 'higher_Nao', 'higher_Sim', 'internet_Nao', 'internet_Sim']
    y = 'G2'
    mse_tree, mae_tree = pred_decision_tree(i, X, y)
    coefs.loc[len(coefs)] = [name, 'Arvore de  Decisao', str(X), str(y), mse_tree, '-', mae_tree]
for name, i in zip(df_names, dfs):
    X = ['failures', 'studytime', 'Medu', 'Fedu', 'school_GP', 'school_MS', 'higher_Nao', 'higher_Sim', 'internet_Nao', 'internet_Sim']
    y = ['G1', 'G2']
    mse_tree, mae_tree = pred_decision_tree(i, X, y)
    coefs.loc[len(coefs)] = [name, 'Arvore de  Decisao', str(X), str(y), mse_tree, '-', mae_tree]
for name, i in zip(df_names, dfs):
    X = ['G1', 'G2']
    y = 'G3'
    mse_tree, mae_tree = pred_decision_tree(i, X, y)
    coefs.loc[len(coefs)] = [name, 'Arvore de  Decisao', str(X), str(y), mse_tree, '-', mae_tree]



for name, i in zip(df_names, dfs):
    X = ['failures', 'studytime', 'Medu', 'Fedu', 'school_GP', 'school_MS', 'higher_Nao', 'higher_Sim', 'internet_Nao', 'internet_Sim']
    y = 'G1'
    mse_gb, mae_gb = pred_gradient_boosting_regression(i, X, y)
    coefs.loc[len(coefs)] = [name, 'Gradient Boosting', str(X), str(y), mse_gb, '-', mae_gb]
for name, i in zip(df_names, dfs):
    X = ['failures', 'studytime', 'Medu', 'Fedu', 'school_GP', 'school_MS', 'higher_Nao', 'higher_Sim', 'internet_Nao', 'internet_Sim']
    y = 'G2'
    mse_gb, mae_gb = pred_gradient_boosting_regression(i, X, y)
    coefs.loc[len(coefs)] = [name, 'Gradient Boosting', str(X), str(y), mse_gb, '-', mae_gb]
for name, i in zip(df_names, dfs):
    X = ['G1', 'G2']
    y = 'G3'
    mse_gb, mae_gb = pred_gradient_boosting_regression(i, X, y)
    coefs.loc[len(coefs)] = [name, 'Gradient Boosting', str(X), str(y), mse_gb, '-', mae_gb]



for name, i in zip(df_names, dfs):
    X = ['failures', 'studytime', 'Medu', 'Fedu', 'school_GP', 'school_MS', 'higher_Nao', 'higher_Sim', 'internet_Nao', 'internet_Sim']
    y = 'G1'
    mse_knn, mae_knn = pred_knn_regression(i, X, y)
    coefs.loc[len(coefs)] = [name, 'KNN', str(X), str(y), mse_knn, '-', mae_knn]
for name, i in zip(df_names, dfs):
    X = ['failures', 'studytime', 'Medu', 'Fedu', 'school_GP', 'school_MS', 'higher_Nao', 'higher_Sim', 'internet_Nao', 'internet_Sim']
    y = 'G1'
    mse_knn, mae_knn = pred_knn_regression(i, X, y)
    coefs.loc[len(coefs)] = [name, 'KNN', str(X), str(y), mse_knn, '-', mae_knn]
for name, i in zip(df_names, dfs):
    X = ['failures', 'studytime', 'Medu', 'Fedu', 'school_GP', 'school_MS', 'higher_Nao', 'higher_Sim', 'internet_Nao', 'internet_Sim']
    y = ['G1', 'G2']
    mse_knn, mae_knn = pred_knn_regression(i, X, y)
    coefs.loc[len(coefs)] = [name, 'KNN', str(X), str(y), mse_knn, '-', mae_knn]
for name, i in zip(df_names, dfs):
    X = ['G1', 'G2']
    y = 'G3'
    mse_knn, mae_knn = pred_knn_regression(i, X, y)
    coefs.loc[len(coefs)] = [name, 'KNN', str(X), str(y), mse_knn, '-', mae_knn]



for name, i in zip(df_names, dfs):
    X = ['failures', 'studytime', 'Medu', 'Fedu', 'school_GP', 'school_MS', 'higher_Nao', 'higher_Sim', 'internet_Nao', 'internet_Sim']
    y = 'G1'
    mse_lgbm, mae_lgbm = pred_light_gbm(i, X, y)
    coefs.loc[len(coefs)] = [name, 'Light GBM', str(X), str(y), mse_lgbm, '-', mae_lgbm]
for name, i in zip(df_names, dfs):
    X = ['failures', 'studytime', 'Medu', 'Fedu', 'school_GP', 'school_MS', 'higher_Nao', 'higher_Sim', 'internet_Nao', 'internet_Sim']
    y = 'G2'
    mse_lgbm, mae_lgbm = pred_light_gbm(i, X, y)
    coefs.loc[len(coefs)] = [name, 'Light GBM', str(X), str(y), mse_lgbm, '-', mae_lgbm]
for name, i in zip(df_names, dfs):
    X = ['G1', 'G2']
    y = 'G3'
    mse_lgbm, mae_lgbm = pred_light_gbm(i, X, y)
    coefs.loc[len(coefs)] = [name, 'Light GBM', str(X), str(y), mse_lgbm, '-', mae_lgbm]

coefs['rmse'] = np.sqrt(coefs['mse'].astype(float))

coefs['rmse'] = coefs['rmse'].apply(lambda x: round(x, 2))
coefs['mse'] = coefs['mse'].apply(lambda x: round(x, 2))
coefs['mae'] = coefs['mae'].apply(lambda x: round(x, 2))

# 2 - Dicionário de Colunas

#### Para o melhor entendimento do que cada uma das categorias significa no contexto da análise, é necessário um dicionário de cada uma das colunas

1) **school**.......................................Escola ("GP" = Gabriel Pereira / "MS" = Mousinho da Silveira)

2) **sex**..............................................Genero ("F" = Feminino / "M" = Masculino)
3) **age**..............................................Idade do aluno
4) **address**.....................................Tipo do endereço ("U" = Urbano / "R" = Rural)
5) **famsize**.....................................Tamanho da Família ("LE3" = 3 Pessoas ou menos / "GT3" = Mais que 3 pessoas)
6) **Pstatus**......................................Estatus dos pais ("J" = Morando juntos / "S" = Separados)
7) **Medu**.........................................Educação da Mãe (0 = Nenhum / 1 = Fundamental / 2 = Quinto até Nono ano / 3 = Colégio / 4 = Ensino Superior)
8) **Fedu**...........................................Educação do Pai (0 = Nenhum / 1 = Fundamental / 2 = Quinto até Nono ano / 3 = Colégio / 4 = Ensino Superior)
9) **Mjob**..........................................Ocupação da Mãe (Professor / Saúde / Func. Público / Lar / Outro)
10) **Fjob**............................................Ocupação do Pai (Professor / Saúde / Func. Público / Lar / Outro)
11) **reason**.......................................Razão para escolher a escola ("Local" = Perto de casa / Reputação / "Curso" = Preferencia de curso / Outro)
12) **guardian**..................................Guarda do estudante
13) **traveltime**...............................Tempo de casa até a escola (1 = Menos que 15 min / 2 = 15 a 30 min / 3 = 30 min a 1 hora / 4 = Maior que 1 hora)
14) **studytime**................................Tempo semanal de estudo (1 = Menos que 2 horas / 2 = 2 a 5 horas / 3 = 5 a 10 horas / 4 = Mais que 10 horas)
15) **failures**......................................Número de vezes que o aluno repetiu de ano
16) **schoolsup**................................Teve suporte educacional extra (Sim ou Não)
17) **famsup**......................................Teve suporte educacional familiar (Sim ou Não)
18) **paid**.............................................Teve aulas extras pagas dentro da disciplina do curso (Matemática ou Português) (Sim ou Não)
19) **activities**...................................Teve atividades extracurriculares (Sim ou Não)
20) **nursery**......................................Frequentou creche (Sim ou Não)
21) **higher**.........................................Deseja cursar ensino superior (Sim ou Não)
22) **internet**.....................................Tem acesso à internet em casa (Sim ou Não)
23) **romantic**...................................Está em um relacionamento amoroso (Sim ou Não)
24) **famrel**........................................Qualidade das relações familiares (De 1 = Muito ruim até 5 = Excelente)
25) **freetime**....................................Tem tempo extra depois da escola (De 1 = Muito baixo até 5 = Muito alto)
26) **goout**.........................................Sai com os amigos (De 1 = Muito baixo até 5 = Muito alto)
27) **Dalc**.............................................Consumo de álcool durante a semana (De 1 = Muito baixo até 5 = Muito alto)
28) **Walc**............................................Consumo de álcool no final de semana (De 1 = Muito baixo até 5 = Muito alto)
29) **health**.........................................Estado de saúde atual (De 1 = Muito baixo até 5 = Muito alto)
30) **absences**...................................Número de faltas na escola
31) **G1**................................................Nota do primeiro trimestre (De 0 a 20)
31) **G2**................................................Nota do segundo trimestre (De 0 a 20)
32) **G3**................................................Nota final (terceiro trimestre) (De 0 a 20)

# 3 - Preparação dos dados

##### Não sendo necessário fazer nenmhum tratamento referente à dados faltantes, podemos tratar as variáveis categóricas

##### Vamos realizar algumas preparações, como abreviações, traduções, mudanças que não causarão impacto à integridade dos dados, a fim de apenas facilitar o entendimento.

In [ ]:
# for i in (dfs):
#     #i['school'] = i['school'].str.replace("GP", str(1)).str.replace("MS", str(0))
#     i['Pstatus'] = i['Pstatus'].str.replace("A", "S").str.replace("T", "J")
#     i['sex'] = i['sex'].str.replace("M", "Masculino").str.replace("F", "Feminino")
#     i['address'] = i['address'].str.replace("R", "Rural").str.replace("U", "Urbano")
#     i['Mjob'] = i['Mjob'].str.replace("at_home", "Lar").str.replace("health", "Saude").str.replace("other", "Outro").str.replace("services", "Func. Publico").str.replace("teacher", "Professor")
#     i['Fjob'] = i['Fjob'].str.replace("at_home", "Lar").str.replace("health", "Saude").str.replace("other", "Outro").str.replace("services", "Func. Publico").str.replace("teacher", "Professor")
#     i['reason'] = i['reason'].str.replace("home", "Local").str.replace("reputation", "Reputacao").str.replace("course", "Curso").str.replace("other", "Outro")
#     i['guardian'] = i['guardian'].str.replace("mother", "Mae").str.replace("father", "Pai").str.replace("other", "Outro")

#     i['schoolsup'] = i['schoolsup'].str.replace("yes", "Sim").str.replace("no", "Nao")
#     i['famsup'] = i['famsup'].str.replace("yes", "Sim").str.replace("no", "Nao")
#     i['paid'] = i['paid'].str.replace("yes", "Sim").str.replace("no", "Nao")
#     i['activities'] = i['activities'].str.replace("yes", "Sim").str.replace("no", "Nao")
#     i['nursery'] = i['nursery'].str.replace("yes", "Sim").str.replace("no", "Nao")
#     i['higher'] = i['higher'].str.replace("yes", "Sim").str.replace("no", "Nao")
#     i['internet'] = i['internet'].str.replace("yes", "Sim").str.replace("no", "Nao")
#     i['romantic'] = i['romantic'].str.replace("yes", "Sim").str.replace("no", "Nao")

#     #i['school'] = i['school'].astype(int)

In [ ]:
df_mat2 = df_mat.copy()
df_por2 = df_por.copy()

df_mat2['school'] = df_mat2['school'].str.replace("GP", str(1)).str.replace("MS", str(0)).astype(int)
df_mat2['schoolsup'] = df_mat2['schoolsup'].str.replace("Sim", str(1)).str.replace("Nao", str(0)).astype(int)
df_mat2['address'] = df_mat2['address'].str.replace("Urbano", str(1)).str.replace("Rural", str(0)).astype(int)
df_mat2['famsize'] = df_mat2['famsize'].str.replace("GT3", str(1)).str.replace("LE3", str(0)).astype(int)

df_por2['school'] = df_por2['school'].str.replace("GP", str(1)).str.replace("MS", str(0)).astype(int)
df_por2['schoolsup'] = df_por2['schoolsup'].str.replace("Sim", str(1)).str.replace("Nao", str(0)).astype(int)
df_por2['address'] = df_por2['address'].str.replace("Urbano", str(1)).str.replace("Rural", str(0)).astype(int)
df_por2['famsize'] = df_por2['famsize'].str.replace("GT3", str(1)).str.replace("LE3", str(0)).astype(int)

# 4 - Uma análise exploratória

### 4.1 - Algumas análises Univariadas

##### Iniciaremos a análise vendo algumas variáveis separadamente, como estes dados mostram notas de provas de alunos, as três perguntas iniciais que queremos fazer são:

1) Qual a proporção de homens e mulheres ?
2) Quantas vezes cada nota foi obtida ?
3) Há alguma variável da qual devemos nos preocupar com *outliers*, ou seja, valores muito fora da curva ?

##### Podemos alcançar as respostas das 2 primeiras perguntas com gráficos de barra simples e histogramas. Já a terceira pergunta, baseado no dicionário das variáveis, como a maioria estão numa escala de 1 a 5, não haverão valores fugindo disto, apenas a variável *"absences"* (número de faltas) poderá nos mostrar algum valor discrepante, para ela, usaremos um gráfico de boxplot.

##### Os gráficos de barra abaixo mostram que para matemática, a quantidade de homens e mulheres são bem parecidas, porém em português, uma quantidade um pouco maior de dados de alunas foi coletado


In [ ]:
sns.set_style("whitegrid")

fig, axes = plt.subplots(1, 2, figsize=(8, 4))

bar_mat = df_mat['sex'].value_counts().reset_index()
percentages = bar_mat['count'] / bar_mat['count'].sum() * 100

bar_mat.plot(kind='bar', x='sex', y='count', ax=axes[0], title='Quantidade por Genero - Matematica', legend=False)

for i in range(0,1):
    for j, percentage in enumerate(percentages):
        axes[i].text(j, percentage, f"{percentage:.2f}%", ha="center", va="bottom")

axes[0].set_xlabel('Genero')
axes[0].set_ylabel('Quantidade')
axes[0].set_xticklabels(["Feminino", "Masculino"], rotation=0)



bar_por = df_por['sex'].value_counts().reset_index()
percentages = bar_por['count'] / bar_por['count'].sum() * 100

bar_por.plot(kind='bar', x='sex', y='count', ax=axes[1], title='Quantidade por Genero - Portugues', legend=False)
axes[1].set_xlabel('Genero')
axes[1].set_ylabel('Quantidade')
axes[1].set_xticklabels(["Feminino", "Masculino"], rotation=0)

for i in range(1,2):
    for j, percentage in enumerate(percentages):
        axes[i].text(j, percentage, f"{percentage:.2f}%", ha="center", va="bottom")


plt.tight_layout(w_pad=3)
plt.show()

print("\t\tGráfico 1 - Barras - Quantidade de cada gênero.")

##### Os 6 histogramas abaixo representam a frequência de cada nota, cada linha representa uma matéria, e cada coluna um trimestre do ano.

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(12, 7))

sns.histplot(df_mat['G1'], kde=True, ax=axes[0][0])
sns.histplot(df_mat['G2'], kde=True, ax=axes[0][1])
sns.histplot(df_mat['G3'], kde=True, ax=axes[0][2])

sns.histplot(df_por['G1'], kde=True, ax=axes[1][0])
sns.histplot(df_por['G2'], kde=True, ax=axes[1][1])
sns.histplot(df_por['G3'], kde=True, ax=axes[1][2])

for i in range(0,2):
    for j in range(0,3):

        if i == 0:
            ylim = 80
            sub = 'Matemática'
        else:
            ylim = 110
            sub = 'Português'

        if j == 0: axes[i][j].set_ylabel("Frequencia")
        else: axes[i][j].set_ylabel("")

        axes[i][j].set_xlabel("Nota "+ str(j+1) +"º Trimestre - " + sub)

        axes[i][j].set_ylim(0, ylim)
        axes[i][j].set_xlim(0, 20)
        axes[i][j].set_xticks(np.arange(0, 21, 2))

for i in range(0, 2):
    for j in range(0, 3):
        label = axes[i][j].get_xticklabels()
        ticks = axes[i][j].get_xticks()
        axes[i][j].set_xticks(ticks)
        axes[i][j].set_xticklabels(label)

plt.tight_layout(h_pad=3, w_pad=2)
plt.show()

print("\t\tGráfico 2 - Histograma - Frequência de cada nota por trimestre.")

##### Podemos concluir que, a quantidade de notas baixas aumentam com o passar do ano para ambas as matérias, porém este aumento é muito mais evidente em matemática. Da qual do 2º para o 3º trimestre, o número de alunos que tiraram nota 0 quase triplicaram.

#### Este aumento significativo em matemática mas não tanto em português, pode ser causado pelos aumento da dificuldade da matéria, ou talvez a metodologia adotada pela escola. Outra análise poderá concluir algo correlacionando isso com alguma outra variável disponível.

#### Abaixo temos um gráfico boxplot da quantidade de faltas dos alunos, com o objetivo de analisar outliers.

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(10, 5))

sns.boxplot(df_mat.absences, orient="h", ax=axes[0])
sns.boxplot(df_por.absences, orient="h", ax=axes[1])

axes[0].set_xlabel("Faltas - Matematica")
axes[1].set_xlabel("Faltas - Portugues")

plt.tight_layout()
plt.show()

print("\t\tGráfico 3 - Boxplot - Outliers de falta.")

#### Podemos tirar duas principais conclusões com os gráficos acima:

1) Alunos faltaram quase duas vezes mais às aulas de matemática do que português.
2) Os alunos que tiveram uma falta muito acima da média, faltaram mais do que o dobro em matemática do que português, ou seja, emquanto temos 2 alunos que faltaram 30 ou 35 vezes me português, temos 5 alunos que faltaram mais de 35, 40, mais de 50 e até mais do que 70 vezes.

#### Essas duas conclusões podem nos ajudar a entender o decaimento das notas de matemática, que embora tanto a nota quanto falta possam consequências de uma outra causa em comum, as faltas tem uma grande chance de ser um fator contribuinte para o decaimento das notas.

## 4.2 - Algumas análises Bivariadas

### 4.2.1 - Relações de Nota vs Algumas variáveis categóricas

##### Tendo uma ideia inicial de como as variáveis se comportam e hipotetizando algumas possíveis causas pra esses comportamentos, vejamos como algumas dessas variáveis se comportam com relação à outras.

##### Para isso, podemos pensar em algumas perguntas iniciais simples, que envolvam a nota e alguma variável categórica e não numérica:

1) Há uma diferença entre notas dependendo de onde o aluno mora ?
2) Há uma diferença entre notas dependendo de se o aluno teve algum suporte ou aula extra ?
3) Há uma diferença entre notas dependendo do tamanho e relação familiar dos alunos ? 
4) Há uma diferença entre notas e a escola ?

##### Vamos analisar algumas dessas questões utilizando gráficos de sino.

In [ ]:
showViolinPlot('school')
print("\t\tGráfico 4 - Violino - Frequência de Notas por Escola.")
print("\n\t\tGP - Gabriel Pereire\n\t\tMS - Mousinho da Silveira")

In [ ]:
showViolinPlot('address')
print("\t\tGráfico 5 - Violino - Frequência de Notas por tipo de endereço.")

In [ ]:
showViolinPlot('traveltime')
print("\t\tGráfico 6 - Violino - Frequência de Notas por tempo de deslocamento.")
print("\n\t\t1 - Menos de 15 minutos\n\t\t2 - 15 a 30 Minutos\n\t\t3 - 30 a 60 minutos\n\t\t4 - Mais de 60 minutos")

In [ ]:
showViolinPlot('schoolsup')
print("\t\tGráfico 7 - Violino - Frequência de Notas por suporte educacional.")

In [ ]:
showViolinPlot('famrel')
print("\t\tGráfico 8 - Violino - Frequência de Notas por relação familiar.")
print("\n\t\t1 - Muito ruim\n\t\t2 - Ruim\n\t\t3 - Normal\n\t\t4 - Boa\n\t\t5 - Excelente")

In [ ]:
showViolinPlot('famsize')
print("\t\tGráfico 9 - Violino - Frequência de Notas por tamanho da família.")
print("\n\t\tLE3 - Menor ou igual a 3 integrantes\n\t\tGT3 - Mais que 3 integrantes")

##### Os gráficos acima nos permitem tirar algumas conclusões:

1) Confirmação de que as notas, em todos os casos, diminuem ao longo do ano, principalmente em matemática.
2) As notas de matemática nas escolas estão relativamente equilibradas, porém em português, há uma grande maioria de notas acima da média na escola Gabriel Pereira, e quase uma inexistência de notas 0, em contrapartida com a escola Mousinho da Silveira *(Gráfico 4)*.
3) Alunos que moram em zona urbana tendem a ter uma concentração maior de notas acima da média e uma concentração menor de notas baixas *(Gráfico 5)*.
4) Alunos que levam mais tempo se deslocando de casa para a escola tendem a ter notas menores *(Gráfico 6)*.
5) Alunos com suporte educacional extra tendem a concentrar suas notas entre 7.5 e 10, enquanto alunos sem suporte tem uma frequencia de notas mais distribuidas. Porém apenas no começo do ano, do meio para frente, há um aumento de notas baixas e 0 para os alunos sem suporte *(Gráfico 7)*.
6) A relação familiar do aluno interfere muito pouco para as suas notas, alunos com uma boa relação familiar até demonstram ter uma frequência de notas baixas um pouco maior do que alunos com uma má relação familiar *(Gráfico 8)*.
7) Alunos com família maior de 3 pessoas tem uma maior concentração te notas baixas do que alunos com família menor *(Gráfico 9)*.

### 4.2.2 - Correlações das Variáveis numéricas

##### Vimos algumas relações das notas com variáveis categóricas, como as notas se relacionam com variáveis numéricas ? Posto isso, há perguntas que são importantes fazermos, por exemplo:

1) Qual a correlação da nota com a educação dos pais ?
2) Qual a correlação da nota com o numero de faltas ou repetições de ano ?
3) Há uma correlação forte entre outras variáveis sem ser nota ? Essa correlação pode explicar alguma das hipóteses estipuladas até agora ?

##### Fazemos isso calculando a correlação entre uma variável e outra, e gerando um número, variando de -1 a 1.

##### Estes números são então colocados em uma matriz, e cada número é pintado de uma cor diferente, onde é possível saber qual variável se relaciona com qual, e se a correlação é forte ou fraca.

##### ***OBS: O número da correlação, quanto mais próximo de 1, significa uma correlação positiva forte, ou seja são diretamente proporcionais, conforme uma variável sobe, a outra também sobe. Quanto mais próximo de -1, significa uma correlação negativa forte, ou seja inversamente proporcionais, conforme uma variável sobe, a outra desce. E quanto mais próximo de 0, significa que há muito pouca ou nenhuma correlação entre as variáveis.***

In [ ]:
showHeatmap(df_mat2, 'Matemática', 'coolwarm', 10)
print("\t\tGráfico 10 - Mapa de Calor - Correlação das variáveis numéricas de Matemática.")

In [ ]:
showHeatmap(df_por2, 'Português', 'turbo', 11)
print("\t\tGráfico 11 - Mapa de Calor - Correlação das variáveis numéricas de Português.")

##### Com os mapas de calor acima, vemos alguns pontos interessantes:

1) Há uma correlação positiva entre a educação dos pais e as notas (entre 0.15 e 0.26).
2) Confirmamos que há uma correlação positiva entre a escola e a nota de português, porém não com à nota de matemática.
3) Há uma correlação negativa entre a educação dos pais e o tempo que um aluno leva para chegar à escola (entre -0.27 a -0.16).
4) Há uma correlação positiva forte entre as notas, alunos que tiraram notas boas em um trimestre dificilmente tiram notas ruins em outro (entre 0.8 e 0.92).
5) *Confirmamos que quase não há correlação entre notas e relação familiar dos alunos, já que a relação familiar também não está correlacionado com quanto tempo o aluno estuda*.


##### Sobre consumo de álcool

*Há uma correlação negativa entre consumo de alcool e as notas, porém quase não há correlação entre nota e quantas vezes o aluno sai com os amigos para as notas em Português. Esta relação se inverte para as notas de Matemática, da qual quase não há correlação entre consumo de alcool e notas, porém há correlação negativa entre nota e quantas vezes o aluno sai*.

##### Sobre tempo de estudo

*Há uma correlação entre quanto tempo o aluno passa estudando e sua nota, ou seja, quanto mais tempo um aluno passa estudando, maior é a nota. Porém, essa correlação é quase o dobro em Português do que em Matemática. Podendo explicar o maior aumento de notas baixas durante o ano, significando que esta matéria fique bastante difícil com relação ao tempo, de maneira que nem um aluno que passa muito tempo estudando consiga um bom desempenho.*

### 4.2.3 - Comportamento das médias

##### Com as observações vistas anteriormente, vamos colocar em alguns gráficos e tabelas, o comportamento da média das notas em relação à quanto tempo o aluno estuda e à educação dos pais.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

study_time_mat = df_mat.groupby(['studytime'])[['G1', 'G2', 'G3']].mean().rename(index={1: '< 2 horas', 2: '2 a 5 horas', 3: '5 a 10 horas', 4: '> 10 horas'})
study_time_por = df_por.groupby(['studytime'])[['G1', 'G2', 'G3']].mean().rename(index={1: '< 2 horas', 2: '2 a 5 horas', 3: '5 a 10 horas', 4: '> 10 horas'})

groups = [study_time_mat, study_time_por]

for i in range(0,2):
    
    if i%2 == 0: title = 'Matemática'
    else: title = 'Português'
    
    groups[i][['G1', 'G2', 'G3']].plot(kind='line', ax=axes[i], title=title)
    axes[i].set_xlabel('Tempo de estudo')
    axes[i].set_ylabel('Média das Notas')
    axes[i].grid(False)

for j in range(0,2):

    label = axes[j].get_xticklabels()
    ticks = axes[j].get_xticks()
    axes[j].set_xticks(ticks)
    axes[j].set_xticklabels(label)

plt.tight_layout()
plt.show()

print("\t\tGráfico 12 - Linha - Tempo de Estudo por Média de nota.")

##### O gráfico acima nos mostra que há um aumento na nota dos alunos que gastam mais tempo estudando, porém há comumente um decréscimo nas notas dos alunos que estudam mais de 10 horas.

##### *Isso explica a diminuição na correlação, porém, com estes dados, não é possível pontuar com exatidão a causa para o fato dos alunos que mais estudam, diminuirem as notas.*

##### Analisemos agora as notas com relação ao nivel educacional dos pais. Abaixo uma tabela com a média das notas de Matemática para cada nível de educação do Pai.

In [ ]:
father_ed_prep_mat = df_mat.groupby(['Fedu'])[['G1', 'G2', 'G3']].mean().rename(index={0: 'Sem escolaridade', 1: 'Ensino Fundamental', 2: 'Ginásio', 3: 'Colégio', 4: 'Ensino Superior'})
father_ed_prep_mat

##### Esta tabela nos mostra que conforme o nível educacional dos pais sobe, a nota média dos alunos aumenta um pouco, porém, há um ponto interessante, vemos que a maior média é justamente obtida por alunos cujo os pais não tem nenhuma escolaridade, por que isso ocorre ?

##### Como a média é um número exato, podemos supor que há apenas 2 ou 1 alunos nesta condição, logo, com uma amostra pequena, irregularidades podem ocorrer, vamos plotar isso em uns gráficos, porém não só de matemática e educação do pai, mas sim todos:

In [ ]:
fig, axes = plt.subplots(4, 2, figsize=(12, 16))

bar_father_mat = df_mat['Fedu'].value_counts().reset_index()
bar_father_por = df_por['Fedu'].value_counts().reset_index()
bar_mother_mat = df_mat['Medu'].value_counts().reset_index()
bar_mother_por = df_por['Medu'].value_counts().reset_index()
bar_father_mat['Fedu'] = bar_father_mat['Fedu'].replace({0: 'Nenhum', 1: 'Fundamental', 2: 'Ginásio', 3: 'Colégio', 4: 'Superior'})
bar_father_por['Fedu'] = bar_father_por['Fedu'].replace({0: 'Nenhum', 1: 'Fundamental', 2: 'Ginásio', 3: 'Colégio', 4: 'Superior'})
bar_mother_mat['Medu'] = bar_mother_mat['Medu'].replace({0: 'Nenhum', 1: 'Fundamental', 2: 'Ginásio', 3: 'Colégio', 4: 'Superior'})
bar_mother_por['Medu'] = bar_mother_por['Medu'].replace({0: 'Nenhum', 1: 'Fundamental', 2: 'Ginásio', 3: 'Colégio', 4: 'Superior'})
father_ed_prep_mat = df_mat.groupby(['Fedu'])[['G1', 'G2', 'G3']].mean().rename(index={0: 'Nenhum', 1: 'Fundamental', 2: 'Ginásio', 3: 'Colégio', 4: 'Superior'})
father_ed_prep_por = df_por.groupby(['Fedu'])[['G1', 'G2', 'G3']].mean().rename(index={0: 'Nenhum', 1: 'Fundamental', 2: 'Ginásio', 3: 'Colégio', 4: 'Superior'})
mother_ed_prep_mat = df_mat.groupby(['Medu'])[['G1', 'G2', 'G3']].mean().rename(index={0: 'Nenhum', 1: 'Fundamental', 2: 'Ginásio', 3: 'Colégio', 4: 'Superior'})
mother_ed_prep_por = df_por.groupby(['Medu'])[['G1', 'G2', 'G3']].mean().rename(index={0: 'Nenhum', 1: 'Fundamental', 2: 'Ginásio', 3: 'Colégio', 4: 'Superior'})

bars = [bar_father_mat, bar_father_por, bar_mother_mat, bar_mother_por]
groups = [father_ed_prep_mat, father_ed_prep_por, mother_ed_prep_mat, mother_ed_prep_por]

for i in range(0,4):

    if i in [0,1]:
        parent = 'Fedu'
        label = 'Educação do Pai'
    else:
        parent = 'Medu'
        label = 'Educação da Mãe'
    
    if i%2 == 0: title = 'Matemática'
    else: title = 'Português'
    
    bars[i].plot(kind='bar', x=parent, y='count', ax=axes[i][0], title=title, legend=False)
    axes[i][0].set_xlabel(label)
    axes[i][0].set_ylabel('Quantidade')
    axes[i][0].grid(False)

    for x, value in enumerate(bars[i]['count']):
        axes[i][0].text(x, value, str(value), ha="center", va="bottom")


    groups[i][['G1', 'G2', 'G3']].plot(kind='line', ax=axes[i][1], title=title)
    axes[i][1].set_xlabel(label)
    axes[i][1].set_ylabel('Média')
    axes[i][1].grid(False)

for i in range(0,4):
    for j in range(0,2):

        if j == 1: angle = 30
        else: angle = 30

        label = axes[i][j].get_xticklabels()
        ticks = axes[i][j].get_xticks()
        axes[i][j].set_xticks(ticks)
        axes[i][j].set_xticklabels(label, rotation=angle)

plt.tight_layout(h_pad=4)
plt.show()

print("\t\tGráfico 13 - Barras e Linha - Quantidade de Cada educação de Pai/Mãe e Média de nota por Educação dos pais.")

##### Os gráficos acima corroboram a informação de que uma amostra pequena está elevando a média. Apenas entre 3 e 7 alunos tem pais sem nenhuma escolaridade. Estes pouco alunos coincidentemente tiraram uma nota mais alta, o que fizeram elevar a nota média deste grupo, explicando essa anormalidade.

##### É similar aos casos onde apenas 1 salário de um diretor ou presidente de empresa eleva a média salarial da empresa para um patamar muito alto, não condizente com o salário da grande maioria dos funcionários.

##### Tirando isso, há uma consistência nos dados, em que quanto maior o nível educacional de um pai ou mãe, em média um pouco maior será a nota dos filhos, as notas dos 3 trimestres crescem de maneira quase uniforme, não tendo nenhuma discrepância entre elas.

# 5 - Considerações finais.

### 5.1 - Conclusões e hipóteses.

##### Ao final de tudo, podemos concluir que as notas dos alunos decaem ao longo do ano, este decaimento é mais evidente em Matemática

Isso explica a diminuição na correlação, porém, com estes dados, não é possível pontuar com exatidão a causa para o fato dos alunos que mais estudam, diminuirem as notas.

Como o alcool influencia negativamente nas notas de português, mas não tanto em matemática, o alcool está correlacionado à quanto tempo livre e quantas vezes o aluno sai, e quando somamos ao fato de que, para português, quanto mais tempo um aluno estuda maior a sua nota, porém o tempo de estudo não impacta tanto na nota de matemática.

Tudo isso nos leva a crer que, há um menor tempo de dedicação aos estudos em matemática, e também há uma crescente complexidade na matéria ao longo do ano.

### 5.2 - Propostas.

##### A escola pode implementar a seguinte medida para combater estes problemas:

1) Aumentar interesse do aluno
2) Recompensar esforço

##### Aumentar o interesse do aluno em estudar matemática ao diminuir o foco em decorar fórmulas onde isso ocorre, ao invés, trazer problemas do ambiente do aluno para a sala de aula, mostrando onde tais numeros, fórmulas e metodologias se aplicam ao cotidiano do contexto em que o aluno está inserido, fazendo-o perceber como a matemática se aplica ao seu próprio dia a dia.

##### Recompensar o esforço do aluno, diminuindo um pouco a quantidade e complexidade do conteúdo, ao decorar menos fórmulas e entender mais, o aluno se sente mais recompensado, e consequentemente mais motivado à estudar mais, quando vê que seu entendimento foi convertido em uma nota maior.

FIM ENTREGA 1

Importação dos DataSets

In [ ]:
# df_mat = pd.read_csv("data/student-mat.csv", sep=';')
# df_por = pd.read_csv("data/student-por.csv", sep=';')
# dfs = [df_mat, df_por]

Tratamento dos dados (tradução e One Hot Encoding)

In [ ]:
# for i in (dfs):
#     i['Pstatus'] = i['Pstatus'].str.replace("A", "S").str.replace("T", "J")
#     i['sex'] = i['sex'].str.replace("M", "Masculino").str.replace("F", "Feminino")
#     i['address'] = i['address'].str.replace("R", "Rural").str.replace("U", "Urbano")
#     i['Mjob'] = i['Mjob'].str.replace("at_home", "Lar").str.replace("health", "Saude").str.replace("other", "Outro").str.replace("services", "Func. Publico").str.replace("teacher", "Professor")
#     i['Fjob'] = i['Fjob'].str.replace("at_home", "Lar").str.replace("health", "Saude").str.replace("other", "Outro").str.replace("services", "Func. Publico").str.replace("teacher", "Professor")
#     i['reason'] = i['reason'].str.replace("home", "Local").str.replace("reputation", "Reputacao").str.replace("course", "Curso").str.replace("other", "Outro")
#     i['guardian'] = i['guardian'].str.replace("mother", "Mae").str.replace("father", "Pai").str.replace("other", "Outro")

#     i['schoolsup'] = i['schoolsup'].str.replace("yes", "Sim").str.replace("no", "Nao")
#     i['famsup'] = i['famsup'].str.replace("yes", "Sim").str.replace("no", "Nao")
#     i['paid'] = i['paid'].str.replace("yes", "Sim").str.replace("no", "Nao")
#     i['activities'] = i['activities'].str.replace("yes", "Sim").str.replace("no", "Nao")
#     i['nursery'] = i['nursery'].str.replace("yes", "Sim").str.replace("no", "Nao")
#     i['higher'] = i['higher'].str.replace("yes", "Sim").str.replace("no", "Nao")
#     i['internet'] = i['internet'].str.replace("yes", "Sim").str.replace("no", "Nao")
#     i['romantic'] = i['romantic'].str.replace("yes", "Sim").str.replace("no", "Nao")

In [ ]:
dfs[0] = pd.get_dummies(df_mat, columns=['school', 'higher', 'internet'])
dfs[1] = pd.get_dummies(df_por, columns=['school', 'higher', 'internet'])

for i in dfs:
    i['school_GP'] = i['school_GP'].astype(int)
    i['school_MS'] = i['school_MS'].astype(int)
    i['higher_Nao'] = i['higher_Nao'].astype(int)
    i['higher_Sim'] = i['higher_Sim'].astype(int)
    i['internet_Nao'] = i['internet_Nao'].astype(int)
    i['internet_Sim'] = i['internet_Sim'].astype(int)

df_mat = dfs[0].copy()
df_por = dfs[1].copy()

# 7 - Planejamento dos Modelos

##### Com os dados analisados e algumas conclusões feitas, o próximo passo é utilizar modelos de Machine Learning com o objetivo de prever as notas dos alunos com base em algumas características.

##### Por exemplo, vimos que tempo de estudo e escola estão correlacionadas com as notas, com base nisso, o quão bem podemos prever a nota que um aluno vai ter, se soubermos apenas o quanto tempo ele estuda e qual escola ele estuda ? Esta é o principal questionamento que vamos responder nesta etapa.

### 7.1 - O que queremos prever de fato ?

##### Queremos prever a variável G1 e G2 (nota do 1º e 2º Trimestre, respectivamente) com base em algumas variáveis, e também confirmar o quão bem podemos prever a variável G3 (nota final) com base nas outras 2 notas.

#### Variáveis a serem usadas nos modelos:
- *failures* - Quantas vezes o aluno repetiu de ano.
- *studytime* - Quantas horas o aluno passa estudando.
- *school* - Qual escola o aluno estuda.
- *Fedu* / *Medu* - Nível de educação da mãe e do pai.
- *higher* - Se o aluno deseja ingressar num Ensino Superior.
- *internet* - Se o aluno tem acesso à internet.

#### Variáveis que não serão usadas nos modelos:
- *traveltime* - Tempo que o aluno gasta se deslocando até a escola.
- *famrel* - O quão boa é a relação familiar do aluno.
- *goout* - O quão frequente ele sai com os amigos.
- *health* - Saúde do aluno.
- *absences* - Quantas vezes o aluno faltou.
- *schoolsup* - Se o aluno teve suporte escolar.
- *address* - Tipo de endereço que o aluno mora.
- *famsize* - Tamanho da família.

### 7.2 - O que será feito, e como ?

##### Vamos utilizar Modelos Supervisionados para previsão, isto é, um modelo onde já temos a resposta (no caso as notas), dividiremos os dados em 2 blocos: Treino e Teste, um bloco será usado para treinar o algoritmo, e o outro bloco será usado para testarmos o quão precisa foram as previsões do modelo.

#### Modelos a serem utilizados:
- *Regressão Linear*
- *Árvore de Decisão*
- *K - Vizinhos mais Próximos*
- *Gradient Boosting*
- *Light GBM*

### 7.3 - Expectativa e próximos passos

##### Fazendo tudo isso, esperamos que os modelos identifiquem fatores-chave e apresentem uma precisão razoável na previsão das notas finais. Iremos captar as métricas e medições de acurácia dos modelos, para podermos avaliar qual deles está performando melhor e sendo mais pertinente para a nossa análise.

# 8 - Relação da EDA com a escolha das variáveis

##### Como foi visto na Análise Exploratória anteriormente, podemos fazer algumas seleções de quais features iremos incluir ou remover.

##### No item 2.1 acima, elencamos as variáveis à serem usadas: *failures* / *studytime* / *school* / *Fedu* / *Medu*

##### Vimos que essas variáveis tem uma correlação positiva com as notas, então serão incluídas nos modelos.

### Através das matrizes abaixo, alguns pontos muito importantes devem ser apontados:

- Alunos com o intuito de ingressar em um ensino superior geralmente tem uma nota um pouco maior.
- Alunos da escola Gabriel Pereira geralmente tem uma nota maior do que os alunos da escola Mousinho da Silveira.
- Coincidentemente, a escola Gabriel Pereira tem uma quantidade de alunos com acesso à internet maior do que a escola Mousinho da Silveira.

### Por causa destas observações, as variáveis *higher* / *internet* também serão inclusas nos modelos.

In [ ]:
df_mat_corr = df_mat.drop(columns=['age', 'address', 'famsize', 'Medu', 'Fedu', 'traveltime', 'studytime', 'failures', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences'])
df_por_corr = df_por.drop(columns=['age', 'address', 'famsize', 'Medu', 'Fedu', 'traveltime', 'studytime', 'failures', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences'])
corr_arr = [df_mat_corr, df_por_corr]
index = [1, 2]

for c, n, i in zip(corr_arr, df_names, index):
    showHeatmap(c, n, 'coolwarm', i)


# 9 - Implementação dos Modelos

##### Os modelos a serem treinados são os que já foram mencionados no item 2.2

##### Abaixo contem o código que os executa

##### Importante observar que em cada execução, é gerada um resultado e métricas de cada um, que são adicionados em uma tabela.

##### Essa tabela será o principal objeto de estudo na próxima etapa desta análise, onde iremos dar uma olhada mais afundo nessas métricas, o que elas significam, saber qual algoritmo conseguiu performar melhor e fazer as melhores previsões, assim como o oposto.

### 9.1 - Regressão Linear

In [ ]:
for name, i in zip(df_names, dfs):
    X = ['failures', 'studytime', 'Medu', 'Fedu', 'school_GP', 'school_MS', 'higher_Nao', 'higher_Sim', 'internet_Nao', 'internet_Sim']
    y = 'G1'
    mse_lr, r2_lr = pred_linear_regression(i, X, y)
    coefs.loc[len(coefs)] = [name, 'Regressao Linear', str(X), str(y), f'{mse_lr:.2f}', f'{r2_lr:.2f}']

In [ ]:
for name, i in zip(df_names, dfs):
    X = ['failures', 'studytime', 'Medu', 'Fedu', 'school_GP', 'school_MS', 'higher_Nao', 'higher_Sim', 'internet_Nao', 'internet_Sim']
    y = 'G2'
    mse_lr, r2_lr = pred_linear_regression(i, X, y)
    coefs.loc[len(coefs)] = [name, 'Regressao Linear', str(X), str(y), f'{mse_lr:.2f}', f'{r2_lr:.2f}']

In [ ]:
for name, i in zip(df_names, dfs):
    X = ['failures', 'studytime', 'Medu', 'Fedu', 'school_GP', 'school_MS', 'higher_Nao', 'higher_Sim', 'internet_Nao', 'internet_Sim']
    y = ['G1', 'G2']
    mse_lr, r2_lr = pred_linear_regression(i, X, y)
    coefs.loc[len(coefs)] = [name, 'Regressao Linear', str(X), str(y), f'{mse_lr:.2f}', f'{r2_lr:.2f}']

In [ ]:
for name, i in zip(df_names, dfs):
    X = ['G1', 'G2']
    y = 'G3'
    mse_lr, r2_lr = pred_linear_regression(i, X, y)
    coefs.loc[len(coefs)] = [name, 'Regressao Linear', str(X), str(y), f'{mse_lr:.2f}', f'{r2_lr:.2f}']

### 9.2 - Árvore de Decisão

In [ ]:
for name, i in zip(df_names, dfs):
    X = ['failures', 'studytime', 'Medu', 'Fedu', 'school_GP', 'school_MS', 'higher_Nao', 'higher_Sim', 'internet_Nao', 'internet_Sim']
    y = 'G1'
    mse_tree = pred_decision_tree(i, X, y)
    coefs.loc[len(coefs)] = [name, 'Arvore de  Decisao', str(X), str(y), f'{mse_tree:.2f}', '-']

In [ ]:
for name, i in zip(df_names, dfs):
    X = ['failures', 'studytime', 'Medu', 'Fedu', 'school_GP', 'school_MS', 'higher_Nao', 'higher_Sim', 'internet_Nao', 'internet_Sim']
    y = 'G2'
    mse_tree = pred_decision_tree(i, X, y)
    coefs.loc[len(coefs)] = [name, 'Arvore de  Decisao', str(X), str(y), f'{mse_tree:.2f}', '-']

In [ ]:
for name, i in zip(df_names, dfs):
    X = ['failures', 'studytime', 'Medu', 'Fedu', 'school_GP', 'school_MS', 'higher_Nao', 'higher_Sim', 'internet_Nao', 'internet_Sim']
    y = ['G1', 'G2']
    mse_tree = pred_decision_tree(i, X, y)
    coefs.loc[len(coefs)] = [name, 'Arvore de  Decisao', str(X), str(y), f'{mse_tree:.2f}', '-']

In [ ]:
for name, i in zip(df_names, dfs):
    X = ['G1', 'G2']
    y = 'G3'
    mse_tree = pred_decision_tree(i, X, y)
    coefs.loc[len(coefs)] = [name, 'Arvore de  Decisao', str(X), str(y), f'{mse_tree:.2f}', '-']

### 9.3 - KNN (K-Vizinhos mais Próximos)

In [ ]:
for name, i in zip(df_names, dfs):
    X = ['failures', 'studytime', 'Medu', 'Fedu', 'school_GP', 'school_MS', 'higher_Nao', 'higher_Sim', 'internet_Nao', 'internet_Sim']
    y = 'G1'
    mse_knn = pred_knn_regression(i, X, y)
    coefs.loc[len(coefs)] = [name, 'KNN', str(X), str(y), f'{mse_knn:.2f}', '-']

In [ ]:
for name, i in zip(df_names, dfs):
    X = ['failures', 'studytime', 'Medu', 'Fedu', 'school_GP', 'school_MS', 'higher_Nao', 'higher_Sim', 'internet_Nao', 'internet_Sim']
    y = 'G1'
    mse_knn = pred_knn_regression(i, X, y)
    coefs.loc[len(coefs)] = [name, 'KNN', str(X), str(y), f'{mse_knn:.2f}', '-']

In [ ]:
for name, i in zip(df_names, dfs):
    X = ['failures', 'studytime', 'Medu', 'Fedu', 'school_GP', 'school_MS', 'higher_Nao', 'higher_Sim', 'internet_Nao', 'internet_Sim']
    y = ['G1', 'G2']
    mse_knn = pred_knn_regression(i, X, y)
    coefs.loc[len(coefs)] = [name, 'KNN', str(X), str(y), f'{mse_knn:.2f}', '-']

In [ ]:
for name, i in zip(df_names, dfs):
    X = ['G1', 'G2']
    y = 'G3'
    mse_knn = pred_knn_regression(i, X, y)
    coefs.loc[len(coefs)] = [name, 'KNN', str(X), str(y), f'{mse_knn:.2f}', '-']

### 9.4 - Gradient Boosting

In [ ]:
for name, i in zip(df_names, dfs):
    X = ['failures', 'studytime', 'Medu', 'Fedu', 'school_GP', 'school_MS', 'higher_Nao', 'higher_Sim', 'internet_Nao', 'internet_Sim']
    y = 'G1'
    mse_gb = pred_gradient_boosting_regression(i, X, y)
    coefs.loc[len(coefs)] = [name, 'Gradient Boosting', str(X), str(y), f'{mse_gb:.2f}', '-']

In [ ]:
for name, i in zip(df_names, dfs):
    X = ['failures', 'studytime', 'Medu', 'Fedu', 'school_GP', 'school_MS', 'higher_Nao', 'higher_Sim', 'internet_Nao', 'internet_Sim']
    y = 'G2'
    mse_gb = pred_gradient_boosting_regression(i, X, y)
    coefs.loc[len(coefs)] = [name, 'Gradient Boosting', str(X), str(y), f'{mse_gb:.2f}', '-']

In [ ]:
for name, i in zip(df_names, dfs):
    X = ['G1', 'G2']
    y = 'G3'
    mse_gb = pred_gradient_boosting_regression(i, X, y)
    coefs.loc[len(coefs)] = [name, 'Gradient Boosting', str(X), str(y), f'{mse_gb:.2f}', '-']

### 9.5 - Light GBM

In [ ]:
for name, i in zip(df_names, dfs):
    X = ['failures', 'studytime', 'Medu', 'Fedu', 'school_GP', 'school_MS', 'higher_Nao', 'higher_Sim', 'internet_Nao', 'internet_Sim']
    y = 'G1'
    mse_lgbm = pred_light_gbm(i, X, y)
    coefs.loc[len(coefs)] = [name, 'Light GBM', str(X), str(y), f'{mse_lgbm:.2f}', '-']

In [ ]:
for name, i in zip(df_names, dfs):
    X = ['failures', 'studytime', 'Medu', 'Fedu', 'school_GP', 'school_MS', 'higher_Nao', 'higher_Sim', 'internet_Nao', 'internet_Sim']
    y = 'G2'
    mse_lgbm = pred_light_gbm(i, X, y)
    coefs.loc[len(coefs)] = [name, 'Light GBM', str(X), str(y), f'{mse_lgbm:.2f}', '-']

In [ ]:
for name, i in zip(df_names, dfs):
    X = ['G1', 'G2']
    y = 'G3'
    mse_lgbm = pred_light_gbm(i, X, y)
    coefs.loc[len(coefs)] = [name, 'Light GBM', str(X), str(y), f'{mse_lgbm:.2f}', '-']

# 10 - Resultados e Métricas

In [ ]:
display(coefs)
print("\t\tImagem 3 - Tabela - Resultados e métricas dos Modelos treinados.")

fim entrega 2

# 11 - Métricas utilizadas

#### Como visto anteriormente, dar uma olhada mais afundo nas métricas, o que elas significam, saber qual algoritmo conseguiu performar melhor e fazer as melhores previsões, assim como o oposto.

#### As métricas que vamos utilizar são:

- *MSE - Erro quadrático médio*
- *RMSE - Raiz do erro quadrático médio*
- *R² - Coeficiente de determinação*
- *MAE - Erro médio absoluto*

### 11.1 - MSE - Erro quadrático médio

##### Avalia o tamanho dos erros que o algoritmo está cometendo e os eleva ao quadrado, a fim de tornar os erros grandes ainda maiores.

##### Este modelo nos ajuda a ver se o modelo está cometendo muitos erros grandes.

### 11.2 - RMSE - Raiz do erro quadrático médio

##### Avalia o tamanho dos erros que o algoritmo está cometendo mas não os eleva ao quadrado.

##### Este modelo nos ajuda a ver se o modelo está cometendo erros como o MSE porém estes erros estarão na mesma unidade da medição.

### 11.3 - R² - Coeficiente de determinação (Regressão Linear)

##### Avalia o quão bem a Regressão Linear consegue explicar alguma coisa baseado em outras. Varia de 0 a 1, correspondendo à 0% e 100%

##### Um R² de 0 significa que o modelo de Regressão Linear não explica nada e essencialmente é como se estivesse adivinhando.

### 11.4 - MAE - Erro médio absoluto

##### O quão longe, em média, as previsões do modelo estão dos valores reais, ignorando se o valor é negativo ou positivo.

# 12 - Tabela de métricas

In [ ]:
coefs

### A tabela acima nos mostra o seguinte:

##### Para todos os modelos utilizados, quando utilizamos as variáveis: *repetições* / *tempo de estudo* / *escola* / *educação mãe* / *educação pai* / *interesse em ensino superior* / *acesso à internet*, para prever as notas G1 e G2. Vemos que o algoritmo comete muitos erros, e erros grandes.

##### O MSE varia entre 7 para Português e 15 (chegando até 19) para Matemática. Porém considerando que a variável alvo varia entre 0 e 20. Um erro de MSE de 7 é bastante considerável.

##### A Regressão Linear consegue prever aproximadamente 15 a 20% das notas dos alunos baseada nessas informações, e o acerto da previsão da nota final baseado nas outras notas é de aproximadamente 80%

##### Por outro lado, considerando valores absolutos do MAE, observamos que a média dos erros do modelo não está tão longe dos valores reais. Eem média os maiores erros foram entre 2 e 3,5. Considerando a escala da variável entre 0 e 20, esta métrica mostrou que os modelos, quando não erraram bastante, tiveram uma taxa de acerto relativamente boa.

# Conclusões Finais

### Com tudo o que vimos nos dados dos estudantes, podemos tirar algumas conclusões:

1) Quanto mais tempo o aluno passa estudando, maior é a nota. Essa correlação é quase o dobro em Português do que em Matemática.

2) Isso explica o aumento de notas baixas durante o ano, significando que esta matéria fique bastante difícil com relação ao tempo, de maneira que nem um aluno que passa muito tempo estudando consiga um bom desempenho.

3) Há um menor tempo de dedicação aos estudos em matemática, por causa da crescente complexidade na matéria ao longo do ano, isso pode fazer com que os alunos acabem se "desmotivando" para estudar.

4) Alunos da escola Gabriel Pereira tem uma nota em português maior do que os alunos da escola Mousinho da Silveira.

5) A estrutura familiar do aluno não se relaciona tanto com a quantidade de horas que o aluno estuda e suas notas.

6) Alunos da zona urbana tem uma média de notas superior à alunos que moram em área rual. Assim como os que estudam mais longe de casa tem uma nota menor dos que estudam mais perto de casa.

7) Alunos com suporte educacional extra tendem a concentrar suas notas entre 7.5 e 10, alunos sem suporte adquirem uma frequência maior de notas 0.

# Propostas

##### Propomos, então, a implementação dos seguintes modelos de modo a mitigar os principais problemas apresentados acima:

## 1 - Aumentar interesse do aluno.

##### Aumentar o interesse do aluno em estudar matemática ao diminuir o foco em decorar fórmulas onde isso ocorre, ao invés, trazer problemas mais palpáveis do ambiente do aluno para a sala de aula, mostrando onde tais numeros, fórmulas e metodologias se aplicam ao cotidiano do contexto em que o aluno está inserido, fazendo-o perceber como a matemática se aplica ao seu próprio dia a dia.

##### Exemplo: como alunos em um contexto rural tem notas menores, podemos ensinar matemática num contexto rural e de campo, mostrando como a matemática se aplica em diferentes áreas e ciclos de plantio, como a matemática foi usada na construção de casas na época dos grandes latifúndios, conceitos de matemática e física que se aplicam à GPS, ferramenta muito usada no controle de máquinas agrícolas.

## 2 - Recompensar esforço.

##### Ao diminuir um pouco a quantidade e complexidade do conteúdo, ao decorar menos fórmulas e entender mais, o aluno se sente mais recompensado, e consequentemente mais motivado à estudar mais, quando vê que seu entendimento foi convertido em uma nota maior.

##### Exemplo: Compor parte da média final com várias lições de casa e trabalhos, pode estimular o aluno à se dedicar apenas um pouco mais todos os dias, aumentando o número de horas que passa estudando sem nem perceber que está estudando, e não apenas motivar o aluno à estudar somente em vésperas de provas.

## 3 - Estudo da escola Gabriel Pereira.

##### Realizar um estudo mais aprofundado e detalhado sobre as diferenças das didáticas e planos estudantis das duas escolas:

- Como são as aulas dos professores ? E suas formações ?
- O quão atualizado é o material didático ? Há diferença entre eles ?
- Há diferenças nos valores da escola ?
- Como estão estruturados os horários e intervalos ?
- Há diferença entre aplicabilidade de provas ?

##### Estes questionamentos devem ser feitos a fim de entender melhor a causa das notas da escola Gabriel Pereira serem melhores, e, partir destes pontos, traçar um plano para a analisar as possíveis implementações e mudanças a serem feitas na escola Mousinho da Silveira.

fim entrega 3

In [ ]:
# def pred_linear_regression(df, X, y):

#     X_train, X_test, y_train, y_test = train_test_split(df[X], df[y], test_size = 0.2, random_state = 42)

#     modelo = LinearRegression()
#     modelo.fit(X_train, y_train)

#     y_pred = modelo.predict(X_test)

#     mse = mean_squared_error(y_test, y_pred)
#     mae = mean_absolute_error(y_test, y_pred)
#     r2 = r2_score(y_test, y_pred)

#     return mse, r2, mae

# def pred_decision_tree(df, X, y):

#     X_train, X_test, y_train, y_test = train_test_split(df[X], df[y], test_size = 0.2, random_state = 42)

#     tree = DecisionTreeRegressor(random_state = 42)
#     tree.fit(X_train, y_train)
#     Y_pred_tree = tree.predict(X_test)
#     mse_tree = mean_squared_error(y_test, Y_pred_tree)
#     mae_tree = mean_absolute_error(y_test, Y_pred_tree)

#     return mse_tree, mae_tree

# def pred_gradient_boosting_regression(df, X, y):

#     X_train, X_test, y_train, y_test = train_test_split(df[X], df[y], test_size = 0.2, random_state = 42)

#     gboost = GradientBoostingRegressor(n_estimators = 100, random_state = 42)
#     gboost.fit(X_train, y_train)

#     Y_pred_gboost = gboost.predict(X_test)

#     mse_gboost = mean_squared_error(y_test, Y_pred_gboost)
#     mae_gboost = mean_absolute_error(y_test, Y_pred_gboost)
#     return mse_gboost, mae_gboost

# def pred_knn_regression(df, X, y):

#     X_train, X_test, y_train, y_test = train_test_split(df[X], df[y], test_size = 0.2, random_state = 42)

#     scaler = StandardScaler()
#     X_train_scaled = scaler.fit_transform(X_train)
#     X_test_scaled = scaler.transform(X_test)

#     knn = KNeighborsRegressor(n_neighbors = 5)
#     knn.fit(X_train_scaled, y_train)

#     Y_pred_knn = knn.predict(X_test_scaled)

#     mse_knn = mean_squared_error(y_test, Y_pred_knn)
#     mae_knn = mean_absolute_error(y_test, Y_pred_knn)
#     return mse_knn, mae_knn

# def pred_light_gbm(df, X, y):

#     X_train, X_test, y_train, y_test = train_test_split(df[X], df[y], test_size = 0.2, random_state = 42)

#     lgb_model = lgbm.LGBMRegressor(n_estimators = 100, random_state = 42, verbosity = -1)
#     lgb_model.fit(X_train, y_train)

#     Y_pred_lgb = lgb_model.predict(X_test)

#     mse_lgb = mean_squared_error(y_test, Y_pred_lgb)
#     mae_lgb = mean_absolute_error(y_test, Y_pred_lgb)
#     return mse_lgb, mae_lgb

# df_names = ['Matemática', 'Português']

# coefs = pd.DataFrame({
#     'materia': [],
#     'modelo': [],
#     'x': [],
#     'variavel_target': [],
#     'mse': [],
#     'r2': [],
#     'mae': []
# })

# df_mat = pd.read_csv("data/student-mat.csv", sep = ';')
# df_por = pd.read_csv("data/student-por.csv", sep = ';')
# dfs = [df_mat, df_por]

# for i in (dfs):
#     i['Pstatus'] = i['Pstatus'].str.replace("A", "S").str.replace("T", "J")
#     i['sex'] = i['sex'].str.replace("M", "Masculino").str.replace("F", "Feminino")
#     i['address'] = i['address'].str.replace("R", "Rural").str.replace("U", "Urbano")
#     i['Mjob'] = i['Mjob'].str.replace("at_home", "Lar").str.replace("health", "Saude").str.replace("other", "Outro").str.replace("services", "Func. Publico").str.replace("teacher", "Professor")
#     i['Fjob'] = i['Fjob'].str.replace("at_home", "Lar").str.replace("health", "Saude").str.replace("other", "Outro").str.replace("services", "Func. Publico").str.replace("teacher", "Professor")
#     i['reason'] = i['reason'].str.replace("home", "Local").str.replace("reputation", "Reputacao").str.replace("course", "Curso").str.replace("other", "Outro")
#     i['guardian'] = i['guardian'].str.replace("mother", "Mae").str.replace("father", "Pai").str.replace("other", "Outro")

#     i['schoolsup'] = i['schoolsup'].str.replace("yes", "Sim").str.replace("no", "Nao")
#     i['famsup'] = i['famsup'].str.replace("yes", "Sim").str.replace("no", "Nao")
#     i['paid'] = i['paid'].str.replace("yes", "Sim").str.replace("no", "Nao")
#     i['activities'] = i['activities'].str.replace("yes", "Sim").str.replace("no", "Nao")
#     i['nursery'] = i['nursery'].str.replace("yes", "Sim").str.replace("no", "Nao")
#     i['higher'] = i['higher'].str.replace("yes", "Sim").str.replace("no", "Nao")
#     i['internet'] = i['internet'].str.replace("yes", "Sim").str.replace("no", "Nao")
#     i['romantic'] = i['romantic'].str.replace("yes", "Sim").str.replace("no", "Nao")

# dfs[0] = pd.get_dummies(df_mat, columns = ['school', 'higher', 'internet'])
# dfs[1] = pd.get_dummies(df_por, columns = ['school', 'higher', 'internet'])

# for i in dfs:
#     i['school_GP'] = i['school_GP'].astype(int)
#     i['school_MS'] = i['school_MS'].astype(int)
#     i['higher_Nao'] = i['higher_Nao'].astype(int)
#     i['higher_Sim'] = i['higher_Sim'].astype(int)
#     i['internet_Nao'] = i['internet_Nao'].astype(int)
#     i['internet_Sim'] = i['internet_Sim'].astype(int)

# for name, i in zip(df_names, dfs):
#     X = ['failures', 'studytime', 'Medu', 'Fedu', 'school_GP', 'school_MS', 'higher_Nao', 'higher_Sim', 'internet_Nao', 'internet_Sim']
#     y = 'G1'
#     mse_lr, r2_lr, mae_lr = pred_linear_regression(i, X, y)
#     coefs.loc[len(coefs)] = [name, 'Regressao Linear', str(X), str(y), mse_lr, f'{r2_lr:.2f}', mae_lr]
# for name, i in zip(df_names, dfs):
#     X = ['failures', 'studytime', 'Medu', 'Fedu', 'school_GP', 'school_MS', 'higher_Nao', 'higher_Sim', 'internet_Nao', 'internet_Sim']
#     y = 'G2'
#     mse_lr, r2_lr, mae_lr = pred_linear_regression(i, X, y)
#     coefs.loc[len(coefs)] = [name, 'Regressao Linear', str(X), str(y), mse_lr, f'{r2_lr:.2f}', mae_lr]
# for name, i in zip(df_names, dfs):
#     X = ['failures', 'studytime', 'Medu', 'Fedu', 'school_GP', 'school_MS', 'higher_Nao', 'higher_Sim', 'internet_Nao', 'internet_Sim']
#     y = ['G1', 'G2']
#     mse_lr, r2_lr, mae_lr = pred_linear_regression(i, X, y)
#     coefs.loc[len(coefs)] = [name, 'Regressao Linear', str(X), str(y), mse_lr, f'{r2_lr:.2f}', mae_lr]
# for name, i in zip(df_names, dfs):
#     X = ['G1', 'G2']
#     y = 'G3'
#     mse_lr, r2_lr, mae_lr = pred_linear_regression(i, X, y)
#     coefs.loc[len(coefs)] = [name, 'Regressao Linear', str(X), str(y), mse_lr, f'{r2_lr:.2f}', mae_lr]



# for name, i in zip(df_names, dfs):
#     X = ['failures', 'studytime', 'Medu', 'Fedu', 'school_GP', 'school_MS', 'higher_Nao', 'higher_Sim', 'internet_Nao', 'internet_Sim']
#     y = 'G1'
#     mse_tree, mae_tree = pred_decision_tree(i, X, y)
#     coefs.loc[len(coefs)] = [name, 'Arvore de  Decisao', str(X), str(y), mse_tree, '-', mae_tree]
# for name, i in zip(df_names, dfs):
#     X = ['failures', 'studytime', 'Medu', 'Fedu', 'school_GP', 'school_MS', 'higher_Nao', 'higher_Sim', 'internet_Nao', 'internet_Sim']
#     y = 'G2'
#     mse_tree, mae_tree = pred_decision_tree(i, X, y)
#     coefs.loc[len(coefs)] = [name, 'Arvore de  Decisao', str(X), str(y), mse_tree, '-', mae_tree]
# for name, i in zip(df_names, dfs):
#     X = ['failures', 'studytime', 'Medu', 'Fedu', 'school_GP', 'school_MS', 'higher_Nao', 'higher_Sim', 'internet_Nao', 'internet_Sim']
#     y = ['G1', 'G2']
#     mse_tree, mae_tree = pred_decision_tree(i, X, y)
#     coefs.loc[len(coefs)] = [name, 'Arvore de  Decisao', str(X), str(y), mse_tree, '-', mae_tree]
# for name, i in zip(df_names, dfs):
#     X = ['G1', 'G2']
#     y = 'G3'
#     mse_tree, mae_tree = pred_decision_tree(i, X, y)
#     coefs.loc[len(coefs)] = [name, 'Arvore de  Decisao', str(X), str(y), mse_tree, '-', mae_tree]



# for name, i in zip(df_names, dfs):
#     X = ['failures', 'studytime', 'Medu', 'Fedu', 'school_GP', 'school_MS', 'higher_Nao', 'higher_Sim', 'internet_Nao', 'internet_Sim']
#     y = 'G1'
#     mse_gb, mae_gb = pred_gradient_boosting_regression(i, X, y)
#     coefs.loc[len(coefs)] = [name, 'Gradient Boosting', str(X), str(y), mse_gb, '-', mae_gb]
# for name, i in zip(df_names, dfs):
#     X = ['failures', 'studytime', 'Medu', 'Fedu', 'school_GP', 'school_MS', 'higher_Nao', 'higher_Sim', 'internet_Nao', 'internet_Sim']
#     y = 'G2'
#     mse_gb, mae_gb = pred_gradient_boosting_regression(i, X, y)
#     coefs.loc[len(coefs)] = [name, 'Gradient Boosting', str(X), str(y), mse_gb, '-', mae_gb]
# for name, i in zip(df_names, dfs):
#     X = ['G1', 'G2']
#     y = 'G3'
#     mse_gb, mae_gb = pred_gradient_boosting_regression(i, X, y)
#     coefs.loc[len(coefs)] = [name, 'Gradient Boosting', str(X), str(y), mse_gb, '-', mae_gb]



# for name, i in zip(df_names, dfs):
#     X = ['failures', 'studytime', 'Medu', 'Fedu', 'school_GP', 'school_MS', 'higher_Nao', 'higher_Sim', 'internet_Nao', 'internet_Sim']
#     y = 'G1'
#     mse_knn, mae_knn = pred_knn_regression(i, X, y)
#     coefs.loc[len(coefs)] = [name, 'KNN', str(X), str(y), mse_knn, '-', mae_knn]
# for name, i in zip(df_names, dfs):
#     X = ['failures', 'studytime', 'Medu', 'Fedu', 'school_GP', 'school_MS', 'higher_Nao', 'higher_Sim', 'internet_Nao', 'internet_Sim']
#     y = 'G1'
#     mse_knn, mae_knn = pred_knn_regression(i, X, y)
#     coefs.loc[len(coefs)] = [name, 'KNN', str(X), str(y), mse_knn, '-', mae_knn]
# for name, i in zip(df_names, dfs):
#     X = ['failures', 'studytime', 'Medu', 'Fedu', 'school_GP', 'school_MS', 'higher_Nao', 'higher_Sim', 'internet_Nao', 'internet_Sim']
#     y = ['G1', 'G2']
#     mse_knn, mae_knn = pred_knn_regression(i, X, y)
#     coefs.loc[len(coefs)] = [name, 'KNN', str(X), str(y), mse_knn, '-', mae_knn]
# for name, i in zip(df_names, dfs):
#     X = ['G1', 'G2']
#     y = 'G3'
#     mse_knn, mae_knn = pred_knn_regression(i, X, y)
#     coefs.loc[len(coefs)] = [name, 'KNN', str(X), str(y), mse_knn, '-', mae_knn]



# for name, i in zip(df_names, dfs):
#     X = ['failures', 'studytime', 'Medu', 'Fedu', 'school_GP', 'school_MS', 'higher_Nao', 'higher_Sim', 'internet_Nao', 'internet_Sim']
#     y = 'G1'
#     mse_lgbm, mae_lgbm = pred_light_gbm(i, X, y)
#     coefs.loc[len(coefs)] = [name, 'Light GBM', str(X), str(y), mse_lgbm, '-', mae_lgbm]
# for name, i in zip(df_names, dfs):
#     X = ['failures', 'studytime', 'Medu', 'Fedu', 'school_GP', 'school_MS', 'higher_Nao', 'higher_Sim', 'internet_Nao', 'internet_Sim']
#     y = 'G2'
#     mse_lgbm, mae_lgbm = pred_light_gbm(i, X, y)
#     coefs.loc[len(coefs)] = [name, 'Light GBM', str(X), str(y), mse_lgbm, '-', mae_lgbm]
# for name, i in zip(df_names, dfs):
#     X = ['G1', 'G2']
#     y = 'G3'
#     mse_lgbm, mae_lgbm = pred_light_gbm(i, X, y)
#     coefs.loc[len(coefs)] = [name, 'Light GBM', str(X), str(y), mse_lgbm, '-', mae_lgbm]

# coefs['rmse'] = np.sqrt(coefs['mse'].astype(float))

# coefs['rmse'] = coefs['rmse'].apply(lambda x: round(x, 2))
# coefs['mse'] = coefs['mse'].apply(lambda x: round(x, 2))
# coefs['mae'] = coefs['mae'].apply(lambda x: round(x, 2))

In [ ]:
def pred_gradient_boosting_regression_grid_search(df, X, y):

    param_grid_gb = {
        'n_estimators': [50, 100, 200],
        'learning_rate': [0.01, 0.1, 0.2],
        'max_depth': [3, 5, 7],
        'subsample': [0.6, 0.8, 1.0]
    }

    grid_search_gb = GridSearchCV(
        estimator = GradientBoostingRegressor(),
        param_grid = param_grid_gb,
        cv = 5,
        scoring = 'neg_mean_absolute_error',
        n_jobs = -1
    )

    X_train, X_test, y_train, y_test = train_test_split(df[X], df[y], test_size = 0.2, random_state = 42)
    grid_search_gb.fit(X_train, y_train)

    return grid_search_gb.best_params_, grid_search_gb.best_score_ * -1


def pred_knn_regression_grid_search(df, X, y):

    param_grid_knn = {
        'n_neighbors': [3, 5, 7, 9],
        'weights': ['uniform', 'distance'],
        'metric': ['euclidean', 'manhattan']
    }

    grid_search_knn = GridSearchCV(
        estimator = KNeighborsRegressor(),
        param_grid = param_grid_knn,
        cv = 5,
        scoring = 'neg_mean_absolute_error',
        n_jobs = -1
    )

    X_train, X_test, y_train, y_test = train_test_split(df[X], df[y], test_size = 0.2, random_state = 42)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    grid_search_knn.fit(X_train_scaled, y_train)

    return grid_search_knn.best_params_, grid_search_knn.best_score_ * -1


def pred_decision_tree_grid_search(df, X, y):

    param_grid_dt = {
        'max_depth': [None, 10, 20],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    }

    grid_search_dt = GridSearchCV(
        estimator = DecisionTreeRegressor(),
        param_grid = param_grid_dt,
        cv = 5,
        scoring = 'neg_mean_absolute_error',
        n_jobs = -1
    )

    X_train, X_test, y_train, y_test = train_test_split(df[X], df[y], test_size = 0.2, random_state = 42)
    grid_search_dt.fit(X_train, y_train)

    return grid_search_dt.best_params_, grid_search_dt.best_score_ * -1


def pred_light_gbm_grid_search(df, X, y):

    param_grid_lgbm = {
        'n_estimators': [50, 100, 200],
        'learning_rate': [0.01, 0.1, 0.2],
        'num_leaves': [20, 31, 40],
        'max_depth': [-1, 10, 20],
        'subsample': [0.6, 0.8, 1.0]
    }

    grid_search_lgbm = GridSearchCV(
        estimator = lgbm.LGBMRegressor(),
        param_grid = param_grid_lgbm,
        cv = 5,  #
        scoring = 'neg_mean_absolute_error',
        n_jobs = -1
    )

    X_train, X_test, y_train, y_test = train_test_split(df[X], df[y], test_size = 0.2, random_state = 42)
    grid_search_lgbm.fit(X_train, y_train)

    return grid_search_lgbm.best_params_, grid_search_lgbm.best_score_ * -1


In [ ]:
coefs2 = pd.DataFrame({
    'materia': [],
    'modelo': [],
    'x': [],
    'variavel_target': [],
    'mae': []
})


for name, i in zip(df_names, dfs):
    X = ['failures', 'studytime', 'Medu', 'Fedu', 'school_GP', 'school_MS', 'higher_Nao', 'higher_Sim', 'internet_Nao', 'internet_Sim']
    y = 'G1'
    best_params_, best_score_ = pred_decision_tree_grid_search(dfs[0], X, y)
    coefs2.loc[len(coefs2)] = [name, 'Arvore de Decisao', str(X), str(y), best_score_]
for name, i in zip(df_names, dfs):
    X = ['failures', 'studytime', 'Medu', 'Fedu', 'school_GP', 'school_MS', 'higher_Nao', 'higher_Sim', 'internet_Nao', 'internet_Sim']
    y = 'G2'
    best_params_, best_score_ = pred_decision_tree_grid_search(dfs[0], X, y)
    coefs2.loc[len(coefs2)] = [name, 'Arvore de Decisao', str(X), str(y), best_score_]
for name, i in zip(df_names, dfs):
    X = ['G1', 'G2']
    y = 'G3'
    best_params_, best_score_ = pred_decision_tree_grid_search(dfs[0], X, y)
    coefs2.loc[len(coefs2)] = [name, 'Arvore de Decisao', str(X), str(y), best_score_]


for name, i in zip(df_names, dfs):
    X = ['failures', 'studytime', 'Medu', 'Fedu', 'school_GP', 'school_MS', 'higher_Nao', 'higher_Sim', 'internet_Nao', 'internet_Sim']
    y = 'G1'
    best_params_, best_score_ = pred_gradient_boosting_regression_grid_search(dfs[0], X, y)
    coefs2.loc[len(coefs2)] = [name, 'Gradient Boosting', str(X), str(y), best_score_]
for name, i in zip(df_names, dfs):
    X = ['failures', 'studytime', 'Medu', 'Fedu', 'school_GP', 'school_MS', 'higher_Nao', 'higher_Sim', 'internet_Nao', 'internet_Sim']
    y = 'G2'
    best_params_, best_score_ = pred_gradient_boosting_regression_grid_search(dfs[0], X, y)
    coefs2.loc[len(coefs2)] = [name, 'Gradient Boosting', str(X), str(y), best_score_]
for name, i in zip(df_names, dfs):
    X = ['G1', 'G2']
    y = 'G3'
    best_params_, best_score_ = pred_gradient_boosting_regression_grid_search(dfs[0], X, y)
    coefs2.loc[len(coefs2)] = [name, 'Gradient Boosting', str(X), str(y), best_score_]



for name, i in zip(df_names, dfs):
    X = ['failures', 'studytime', 'Medu', 'Fedu', 'school_GP', 'school_MS', 'higher_Nao', 'higher_Sim', 'internet_Nao', 'internet_Sim']
    y = 'G1'
    best_params_, best_score_ = pred_knn_regression_grid_search(dfs[0], X, y)
    coefs2.loc[len(coefs2)] = [name, 'KNN', str(X), str(y), best_score_]
for name, i in zip(df_names, dfs):
    X = ['failures', 'studytime', 'Medu', 'Fedu', 'school_GP', 'school_MS', 'higher_Nao', 'higher_Sim', 'internet_Nao', 'internet_Sim']
    y = 'G2'
    best_params_, best_score_ = pred_knn_regression_grid_search(dfs[0], X, y)
    coefs2.loc[len(coefs2)] = [name, 'KNN', str(X), str(y), best_score_]
for name, i in zip(df_names, dfs):
    X = ['G1', 'G2']
    y = 'G3'
    best_params_, best_score_ = pred_knn_regression_grid_search(dfs[0], X, y)
    coefs2.loc[len(coefs2)] = [name, 'KNN', str(X), str(y), best_score_]



for name, i in zip(df_names, dfs):
    X = ['failures', 'studytime', 'Medu', 'Fedu', 'school_GP', 'school_MS', 'higher_Nao', 'higher_Sim', 'internet_Nao', 'internet_Sim']
    y = 'G1'
    best_params_, best_score_ = pred_light_gbm_grid_search(dfs[0], X, y)
    coefs2.loc[len(coefs2)] = [name, 'Light GBM', str(X), str(y), best_score_]
for name, i in zip(df_names, dfs):
    X = ['failures', 'studytime', 'Medu', 'Fedu', 'school_GP', 'school_MS', 'higher_Nao', 'higher_Sim', 'internet_Nao', 'internet_Sim']
    y = 'G2'
    best_params_, best_score_ = pred_light_gbm_grid_search(dfs[0], X, y)
    coefs2.loc[len(coefs2)] = [name, 'Light GBM', str(X), str(y), best_score_]
for name, i in zip(df_names, dfs):
    X = ['G1', 'G2']
    y = 'G3'
    best_params_, best_score_ = pred_light_gbm_grid_search(dfs[0], X, y)
    coefs2.loc[len(coefs2)] = [name, 'Light GBM', str(X), str(y), best_score_]

In [ ]:
def pred_gradient_boosting_regression_random_search(df, X, y):

    param_dist_gb = {
        'n_estimators': [50, 100, 200],
        'learning_rate': np.linspace(0.01, 0.3, 10),
        'max_depth': [3, 5, 7],
        'subsample': [0.6, 0.8, 1.0]
    }

    random_search_gb = RandomizedSearchCV(
        estimator = GradientBoostingRegressor(),
        param_distributions = param_dist_gb,
        n_iter = 20,
        cv = 5,
        scoring = 'neg_mean_squared_error',
        n_jobs = -1,
        random_state = 42
    )

    X_train, X_test, y_train, y_test = train_test_split(df[X], df[y], test_size = 0.2, random_state = 42)
    random_search_gb.fit(X_train, y_train)

    return random_search_gb.best_params_, random_search_gb.best_score_ * -1


def pred_knn_regression_random_search(df, X, y):

    param_dist_knn = {
        'n_estimators': [50, 100, 200],
        'learning_rate': np.linspace(0.01, 0.3, 10),
        'max_depth': [3, 5, 7],
        'subsample': [0.6, 0.8, 1.0]
    }

    random_search_knn = RandomizedSearchCV(
        estimator = GradientBoostingRegressor(),
        param_distributions = param_dist_knn,
        n_iter = 20,
        cv = 5,
        scoring = 'neg_mean_squared_error',
        n_jobs = -1,
        random_state = 42
    )

    X_train, X_test, y_train, y_test = train_test_split(df[X], df[y], test_size = 0.2, random_state = 42)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    random_search_knn.fit(X_train_scaled, y_train)

    return random_search_knn.best_params_, random_search_knn.best_score_ * -1


def pred_decision_tree_random_search(df, X, y):

    param_dist_dt = {
        'n_estimators': [50, 100, 200],
        'learning_rate': np.linspace(0.01, 0.3, 10),
        'max_depth': [3, 5, 7],
        'subsample': [0.6, 0.8, 1.0]
    }

    random_search_dt = RandomizedSearchCV(
        estimator = GradientBoostingRegressor(),
        param_distributions = param_dist_dt,
        n_iter = 20,
        cv = 5,
        scoring = 'neg_mean_squared_error',
        n_jobs = -1,
        random_state = 42
    )

    X_train, X_test, y_train, y_test = train_test_split(df[X], df[y], test_size = 0.2, random_state = 42)
    random_search_dt.fit(X_train, y_train)

    return random_search_dt.best_params_, random_search_dt.best_score_ * -1


def pred_light_gbm_random_search(df, X, y):

    param_dist_lgbm = {
        'n_estimators': [50, 100, 200],
        'learning_rate': np.linspace(0.01, 0.3, 10),
        'max_depth': [3, 5, 7],
        'subsample': [0.6, 0.8, 1.0]
    }

    random_search_lgbm = RandomizedSearchCV(
        estimator = GradientBoostingRegressor(),
        param_distributions = param_dist_lgbm,
        n_iter = 20,
        cv = 5,
        scoring = 'neg_mean_squared_error',
        n_jobs = -1,
        random_state = 42
    )

    X_train, X_test, y_train, y_test = train_test_split(df[X], df[y], test_size = 0.2, random_state = 42)
    random_search_lgbm.fit(X_train, y_train)

    return random_search_lgbm.best_params_, random_search_lgbm.best_score_ * -1

In [ ]:
coefs3 = pd.DataFrame({
    'materia': [],
    'modelo': [],
    'x': [],
    'variavel_target': [],
    'mse': []
})


for name, i in zip(df_names, dfs):
    X = ['failures', 'studytime', 'Medu', 'Fedu', 'school_GP', 'school_MS', 'higher_Nao', 'higher_Sim', 'internet_Nao', 'internet_Sim']
    y = 'G1'
    best_params_, best_score_ = pred_decision_tree_random_search(dfs[0], X, y)
    coefs3.loc[len(coefs3)] = [name, 'Arvore de Decisao', str(X), str(y), best_score_]
for name, i in zip(df_names, dfs):
    X = ['failures', 'studytime', 'Medu', 'Fedu', 'school_GP', 'school_MS', 'higher_Nao', 'higher_Sim', 'internet_Nao', 'internet_Sim']
    y = 'G2'
    best_params_, best_score_ = pred_decision_tree_random_search(dfs[0], X, y)
    coefs3.loc[len(coefs3)] = [name, 'Arvore de Decisao', str(X), str(y), best_score_]
for name, i in zip(df_names, dfs):
    X = ['G1', 'G2']
    y = 'G3'
    best_params_, best_score_ = pred_decision_tree_random_search(dfs[0], X, y)
    coefs3.loc[len(coefs3)] = [name, 'Arvore de Decisao', str(X), str(y), best_score_]


for name, i in zip(df_names, dfs):
    X = ['failures', 'studytime', 'Medu', 'Fedu', 'school_GP', 'school_MS', 'higher_Nao', 'higher_Sim', 'internet_Nao', 'internet_Sim']
    y = 'G1'
    best_params_, best_score_ = pred_gradient_boosting_regression_random_search(dfs[0], X, y)
    coefs3.loc[len(coefs3)] = [name, 'Gradient Boosting', str(X), str(y), best_score_]
for name, i in zip(df_names, dfs):
    X = ['failures', 'studytime', 'Medu', 'Fedu', 'school_GP', 'school_MS', 'higher_Nao', 'higher_Sim', 'internet_Nao', 'internet_Sim']
    y = 'G2'
    best_params_, best_score_ = pred_gradient_boosting_regression_random_search(dfs[0], X, y)
    coefs3.loc[len(coefs3)] = [name, 'Gradient Boosting', str(X), str(y), best_score_]
for name, i in zip(df_names, dfs):
    X = ['G1', 'G2']
    y = 'G3'
    best_params_, best_score_ = pred_gradient_boosting_regression_random_search(dfs[0], X, y)
    coefs3.loc[len(coefs3)] = [name, 'Gradient Boosting', str(X), str(y), best_score_]


for name, i in zip(df_names, dfs):
    X = ['failures', 'studytime', 'Medu', 'Fedu', 'school_GP', 'school_MS', 'higher_Nao', 'higher_Sim', 'internet_Nao', 'internet_Sim']
    y = 'G1'
    best_params_, best_score_ = pred_knn_regression_random_search(dfs[0], X, y)
    coefs3.loc[len(coefs3)] = [name, 'KNN', str(X), str(y), best_score_]
for name, i in zip(df_names, dfs):
    X = ['failures', 'studytime', 'Medu', 'Fedu', 'school_GP', 'school_MS', 'higher_Nao', 'higher_Sim', 'internet_Nao', 'internet_Sim']
    y = 'G2'
    best_params_, best_score_ = pred_knn_regression_random_search(dfs[0], X, y)
    coefs3.loc[len(coefs3)] = [name, 'KNN', str(X), str(y), best_score_]
for name, i in zip(df_names, dfs):
    X = ['G1', 'G2']
    y = 'G3'
    best_params_, best_score_ = pred_knn_regression_random_search(dfs[0], X, y)
    coefs3.loc[len(coefs3)] = [name, 'KNN', str(X), str(y), best_score_]


for name, i in zip(df_names, dfs):
    X = ['failures', 'studytime', 'Medu', 'Fedu', 'school_GP', 'school_MS', 'higher_Nao', 'higher_Sim', 'internet_Nao', 'internet_Sim']
    y = 'G1'
    best_params_, best_score_ = pred_light_gbm_random_search(dfs[0], X, y)
    coefs3.loc[len(coefs3)] = [name, 'Light GBM', str(X), str(y), best_score_]
for name, i in zip(df_names, dfs):
    X = ['failures', 'studytime', 'Medu', 'Fedu', 'school_GP', 'school_MS', 'higher_Nao', 'higher_Sim', 'internet_Nao', 'internet_Sim']
    y = 'G2'
    best_params_, best_score_ = pred_light_gbm_random_search(dfs[0], X, y)
    coefs3.loc[len(coefs3)] = [name, 'Light GBM', str(X), str(y), best_score_]
for name, i in zip(df_names, dfs):
    X = ['G1', 'G2']
    y = 'G3'
    best_params_, best_score_ = pred_light_gbm_random_search(dfs[0], X, y)
    coefs3.loc[len(coefs3)] = [name, 'Light GBM', str(X), str(y), best_score_]


# Tuning de Hiperparametros

##### Vamos aplicar técnicas chamadas de Tuning de Hiperparametros.

##### Isso nada mais é do que ajustes que podemos fazer para calibrar alguns parâmetros de como o algoritmo aplica suas fórmulas.

##### O objetivo disso é vermos se alguns modelos podem ter sua eficiencia melhorada e oferecer melhores resultados, num escopo de diferentes valores.

##### Vamos utilizar 2 técnicas para realizar esses ajustes:

- Grid Search
- Random Search

In [ ]:
coefs2

In [ ]:
coefs3

### As tabelas acima nos mostram o seguinte:

##### A primeira tabela (Referente ao ajuste de Grid Search), podemos observar que em média, temos uma métrica de MAE (Erro médio absoluto) de aproximadamente 2,5, em comparação de erros acima de 3 e quase 4 quando comparados antes dos ajustes.

##### Isso significa que a calibragem do Grid Search ajudou o modelo à performar melhor e fazer predições melhores e com menos erros.

##### A segunda tabela (Referente ao ajuste de Random Search), podemos observar que em média, temos uma métrica de MSE (Erro quadrático médio) de aproximadamente 9 e 13, em comparação de erros de antes dos ajustes.

##### Isso significa que a calibragem do Random Search não ajudou o modelo à performar melhor e fazer predições melhores.

# Conclusões Finais

### Com tudo o que vimos nos dados dos estudantes, podemos tirar algumas conclusões:

1) Quanto mais tempo o aluno passa estudando, maior é a nota. Essa correlação é quase o dobro em Português do que em Matemática.

2) Isso explica o aumento de notas baixas durante o ano, significando que esta matéria fique bastante difícil com relação ao tempo, de maneira que nem um aluno que passa muito tempo estudando consiga um bom desempenho.

3) Há um menor tempo de dedicação aos estudos em matemática, por causa da crescente complexidade na matéria ao longo do ano, isso pode fazer com que os alunos acabem se "desmotivando" para estudar.

4) Alunos da escola Gabriel Pereira tem uma nota em português maior do que os alunos da escola Mousinho da Silveira.

5) A estrutura familiar do aluno não se relaciona tanto com a quantidade de horas que o aluno estuda e suas notas.

6) Alunos da zona urbana tem uma média de notas superior à alunos que moram em área rual. Assim como os que estudam mais longe de casa tem uma nota menor dos que estudam mais perto de casa.

7) Alunos com suporte educacional extra tendem a concentrar suas notas entre 7.5 e 10, alunos sem suporte adquirem uma frequência maior de notas 0.

# Propostas

##### Propomos, então, a implementação dos seguintes modelos de modo a mitigar os principais problemas apresentados acima:

## 1 - Aumentar interesse do aluno.

##### Aumentar o interesse do aluno em estudar matemática ao diminuir o foco em decorar fórmulas onde isso ocorre, ao invés, trazer problemas mais palpáveis do ambiente do aluno para a sala de aula, mostrando onde tais numeros, fórmulas e metodologias se aplicam ao cotidiano do contexto em que o aluno está inserido, fazendo-o perceber como a matemática se aplica ao seu próprio dia a dia.

##### Exemplo: como alunos em um contexto rural tem notas menores, podemos ensinar matemática num contexto rural e de campo, mostrando como a matemática se aplica em diferentes áreas e ciclos de plantio, como a matemática foi usada na construção de casas na época dos grandes latifúndios, conceitos de matemática e física que se aplicam à GPS, ferramenta muito usada no controle de máquinas agrícolas.

## 2 - Recompensar esforço.

##### Ao diminuir um pouco a quantidade e complexidade do conteúdo, ao decorar menos fórmulas e entender mais, o aluno se sente mais recompensado, e consequentemente mais motivado à estudar mais, quando vê que seu entendimento foi convertido em uma nota maior.

##### Exemplo: Compor parte da média final com várias lições de casa e trabalhos, pode estimular o aluno à se dedicar apenas um pouco mais todos os dias, aumentando o número de horas que passa estudando sem nem perceber que está estudando, e não apenas motivar o aluno à estudar somente em vésperas de provas.

## 3 - Estudo da escola Gabriel Pereira.

##### Realizar um estudo mais aprofundado e detalhado sobre as diferenças das didáticas e planos estudantis das duas escolas:

- Como são as aulas dos professores ? E suas formações ?
- O quão atualizado é o material didático ? Há diferença entre eles ?
- Há diferenças nos valores da escola ?
- Como estão estruturados os horários e intervalos ?
- Há diferença entre aplicabilidade de provas ?

##### Estes questionamentos devem ser feitos a fim de entender melhor a causa das notas da escola Gabriel Pereira serem melhores, e, partir destes pontos, traçar um plano para a analisar as possíveis implementações e mudanças a serem feitas na escola Mousinho da Silveira.

fim entrega 4